## Homework 2 warm-up: *How to debug Jupyter notebooks*
Before we start with Homework 2, let's experiment with a useful command to debug in Jupyter notebooks.

When executing an operation in our Jupyter notebook, and we encounter an uncaught exception (as shown below), we can use [%debug](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-debug) to enter into the stack frame where the exception occurred.

Once we are inside the debugger, we can view local variables using `p variable_name`.  We can navigate between stack frames using `up` and `down`.  A full list of python debugger commands can be found [here](https://docs.python.org/3/library/pdb.html#debugger-commands).  When you are done with the debugger type `c` to continue running.

Below, uncomment the `debug_me()` line, and then run the `%debug` statement below it.  Try entering some of the commands in the debugger session:
```
help
p some_variable
p b
ll
up
c
quit
```

In [ ]:
def debug_me():
    some_variable = 'hello world'
    a = 1
    b = 0
    return a / b

# debug_me()  # uncomment this line to try out the debugger

In [ ]:
%debug

# Homework 2 — Efficient Finite-State Methods

Homework 2 focuses on using dynamic programming to compute expected rewards and optimal plans.  The methods explored here are fairly general.  Rather than write task-specific dynamic programming algorithms by hand, we will use *generic* algorithms that are built into the finite-state toolkit, and apply them to *task-specific* weighted finite-state machines.  Thus, our methods will apply broadly — whenever the models and reward functions can be defined using finite-state machines. 

**Chunking:** We are going to be doing a supervised *chunking* task.  "Chunks" are interesting non-empty substrings of the input $\boldsymbol{x}$.  The correct chunks $\boldsymbol{y}$ are known not to overlap with one another, and our prediction $\boldsymbol{a}$ is similarly a set of non-overlapping chunks.  So chunking is sort of like parsing without any recursive structure.  There are $O(J^2)$ possible chunks in a sentence of length $J$, namely the substrings of the form $\boldsymbol{x}_{i:j}$ for $0 \leq i < j \leq J$.  We would like our algorithms to run in $O(J^2)$ as well (rather than the $O(J^3)$ that is needed for the more general problem of context-free parsing).

Specifically, we will consider the task of named entity recognition (NER), in which the chunks refer to "named entities" such as specific persons, places, and organizations.  Treating this as a chunking task means that our annotated data will not be able to mark both of the overlapping named entities in `[The [Johns Hopkins] University]`.  

**Chunking as tagging:** We will encode a chunking as an "IOB" (or "BIO") tagging of the words in each sentence.  Thus, $\boldsymbol{x} \in {\mathcal X}^*$ is a sequence of words, and $\boldsymbol{y} \in {\mathcal Y}^* = \{\text{I,O,B}\}^*$ is a sequence of tags.  The first word of every chunk is tagged with `B` ("beginning"), and the remaining words of that chunk are tagged with `I` ("inside").  All remaining words are tagged with `O` ("outside"). 

Given a sentence of length $J$, the set of legal taggings $\boldsymbol{\mathcal Y}$ is actually a bit smaller than ${\mathcal Y}^\text{J}$.  For example, it does not include `OII` or `III`, because that is not the correct encoding of any set of chunks, and hence could not be decoded.  The restriction is that every `I` must be immediately preceded by either `B` or another `I`.

Note that these are unlabeled chunks.  If we wanted to label each chunk as `PERSON`, `ORGANIZATION`, etc., then we would need a larger tag set ${\mathcal Y}$ to encode this labeled chunking.

**Model:** As our tagging model $p(\boldsymbol{y} \mid \boldsymbol{x})$, we'll use a first-order linear-chain CRF with features that are described below.  Just as for HMMs, inference in this simple model can be performed in $O(J)$ time by the forward-backward algorithm.  We'll discuss some $O(J^2)$ computations later.

**Datasets and Algorithms:** We'll work on the Dutch NER data from the CONLL 2002 Shared Task, in which all words are lowercased (so capitalization features can't be used to help).  We have prepared two different datasets for you:

* The first dataset consists of `train-small` and `dev-small`.  These contain only sentences of length 5 or less and can be used for brute-force training (as in homework 1), since you can enumerate the entire set $\boldsymbol{\mathcal{Y}}$ when computing the gradient ($|\boldsymbol{\mathcal{Y}}| < 3^5 = 243$).

* The second dataset consists of `train`, `dev` and `test`, which contain much longer sequences (up to $45$ words) and thus can **not** be brute forced ($3^{45} = 2954312706550833698643$).  To deal with this larger domain, we will have to use the fact that our scoring function is decomposable, in the sense that we can score possible taggings $\boldsymbol{y}$ and plans $\boldsymbol{a}$ using FSTs.

# Getting set up

In [ ]:
# the current problem size is not large enough to make good use of threads
# set this before we import numpy
import os
os.environ['OMP_NUM_THREADS'] = '1'
import numpy as np
from scipy.misc import logsumexp
import sys
import re
from itertools import islice, product
from pprint import pprint
import math
import csv
from collections import namedtuple   

Data_type = namedtuple('Data', ['xx', 'oo', 'yy'])

def iterate_data(filename='train', *, max_examples=None):
    file = open(filename+'.tsv')
    for n, row in enumerate(csv.DictReader(file, delimiter='\t')):
        if max_examples and n >= max_examples:
            break
        yield Data_type(
            xx=tuple(row['xx'].split()),
            oo=tuple(row['oo'].split()) if 'oo' in row else None,
            yy=tuple(row['yy'].split()) if 'yy' in row else None,
        )

You should copy your classes `TaskSetting`, `ProbabilityModel`, `BoltzmannModel`, `DecisionAgent` from homework 1 into the file `seq2class_homework1.py`.  Currently, that file contains the same stubs that we provided you for homework 1, so feel free to delete everything in that file and replace with your implementations.  (One change: `Probability.params` is now an ordinary attribute; we removed its property getter/setter.)

We will not be re-grading your implementations from homework 1, but if there were any efficiency problems in homework 1, then you may wish to correct those before starting with homework 2.

In [ ]:
try:
    from seq2class_homework1 import (
        TaskSetting,
        ProbabilityModel,
        BoltzmannModel,
        DecisionAgent,
        ViterbiAgent,
        BayesAgent,
        L2LogLikelihood,
        SGDTrainer
    )
except (ImportError, NotImplementedError):
    print('''
    Copy over your implementations from homework 1 into the file `seq2class_homework1.py`.
    This file currently contains the same stubs that you were provided for homework 1.
    ''')
    raise

Let's look at our data.  (Not the test data, of course!)

In [ ]:
print(next(iterate_data('train')))
print("\n",next(iterate_data('dev')))
print("\n",next(iterate_data('dev-small')))

# Integerization

In the previous homework, our input alphabet $\mathcal X$ was fairly small, consisting of only lower and upper case letters.  In this assignment, we will define $\mathcal X$ to consist of entire words instead of just letters.  

It is sometimes helpful to associate each word type in the vocabulary with an unique integer — a technique called "integerization."  This integer index can then be used to help look up counts or parameters associated with that word type.  This is sometimes more convenient then using the word type itself as a dictionary key, for example when dealing with libraries like numpy, PyTorch, and OpenFST that assume integer indices.

Another use of integerization is to associate each parameter in the model with a unique integer.  For example, each $n$-gram feature might be named by an $n$-tuple of strings.  If the tuple's integer index is 7, the corresponding feature weight is stored at `theta[7]`, where `theta` is the parameter vector.

Here's a class for representing vocabularies and parameters in this way.  If you are unfamiliar with the special `__` method names, check [this Python documentation](https://docs.python.org/3/reference/datamodel.html#special-method-names).

In [ ]:
class Integerizer(object):
    """
    A collection of distinct object types, such as a vocabulary or a set of parameter names,
    that are associated with consecutive ints starting at 0.
    """
    
    def __init__(self, iterable=[]):
        """
        Initialize the collection to the empty set, or to the set of *unique* objects in its argument
        (in order of first occurrence).
        """
        # Set up a pair of data structures to convert objects to ints and back again.
        self._objects = []   # list of all unique objects that have been added so far
        self._indices = {}   # maps each object to its integer position in the list
        # Add any objects that were given.
        self.update(iterable)
        
    def __len__(self):
        """
        Number of objects in the collection.
        """
        return len(self._objects)
    
    def __iter__(self):
        """
        Iterate over all the objects in the collection.
        """
        return iter(self._objects)
    
    def __contains__(self, obj):
        """
        Does the collection contain this object?  (Implements `in`.)
        """
        return bool(self.index(obj))
    
    def __getitem__(self, index):
        """
        Return the object with a given index.  
        (Implements subscripting, e.g., `my_integerizer[3]`.)
        """
        return self._objects[index]
        
    def index(self, obj, add=False):
        """
        The integer associated with a given object, or `None` if the object is not in the collection (OOV).  
        Use `add=True` to add the object if it is not present. 
        """
        try:
            return self._indices[obj]
        except KeyError:
            if add:
                # add the object to both data structures
                i = len(self)
                self._objects.append(obj)
                self._indices[obj] = i
                return i
            else:
                return None
            
    def add(self, obj):
        """
        Add the object if it is not already in the collection.
        Similar to `set.add` (or `list.append`).
        """
        self.index(obj, add=True)  # call for side effect; ignore return value
        
    def update(self, iterable):
        """
        Add all the objects if they are not already in the collection.
        Similar to `set.update` (or `list.extend`).
        """
        for obj in iterable:
            self.add(obj) 

Let's try it out.  

In [ ]:
vocab = Integerizer(['','hello','goodbye'])   # 0 represents the empty string '' (epsilon), as in OpenFST
(vocab[2], vocab.index('goodbye'))

In [ ]:
sentence = ('hello','world','if','world','you','be')
[vocab.index(w) for w in sentence]    # includes OOVs

In [ ]:
[vocab.index(w, add=True) for w in sentence]   # expand vocabulary on demand, so no OOVs

In [ ]:
len(vocab)

In [ ]:
'world' in vocab, 'mars' in vocab

# Parameter Dictionaries

We wrote a `ParamDict` class that may be even more convenient.  Instead of mapping parameter names to integer indices, it maps them directly to the parameter values.  This can eliminate the need for `Integerizer`.

In [ ]:
from paramdict import ParamDict   # see paramdict.py for the details
help(ParamDict)

`ParamDict` is an extension of the built-in Python dictionary class (`dict`).  It can be initialized, queried, and modified in the same way.

In [ ]:
params = ParamDict(Noun=2,Verb=3)
params    # note that the default is given by the None entry

In [ ]:
(params['Verb'],params['Noun','violin'])

The main difference is that you can treat `ParamDict` objects as sparse vectors indexed by the parameter names, and do vector arithmetic on them.  Here's what a sparse gradient update to `params` looks like.  Notice that `gradient['Verb']` has the default value of 0, so the entry `params['Verb']` is not affected by the update.

In [ ]:
gradient = ParamDict()
gradient['Noun'],gradient['Noun','violin'] = 5,6
params -= 0.01*gradient
params

*Note:* If you want to be extra-efficient, then you might not want your `ParamDict` keys to include full strings.  Instead, replace larger objects like the strings `'Noun'` and `'violin'` with small fixed-size objects like integers or pointers, which are compact to store and can be very rapidly hashed and compared.  To do this, when you read a string token, convert it to an integer using an `Integerizer`, or convert it to an [interned string](https://en.wikipedia.org/wiki/String_interning) using [`sys.intern`](https://docs.python.org/3/library/sys.html#sys.intern).  Then use tuples of these objects as the `ParamDict` keys for the various model features.  This efficiency trick might be worthwhile in an optimized Cython, C++, or Java implementation. 

# Part 1: Setting up the task

Let's construct `IobTask0`, a particular `TaskSetting` that will work for any chunking problem that is treated with IOB tagging.  All you need to write at this point is the `iterate_y` method (which will be called by `iterate_yy`). 

When defining this method, recall that $\mathcal Y_x$ will contain sequences $\boldsymbol{y}$ that are the same length as our input $\boldsymbol{x}$ and that each word will be labeled with either `B` for the first word of a named entity, `I` for a later word of a named entity, or `O` for a word outside any named entity.  This implies that $\boldsymbol{y}$ should never contain the bigram `OI`, nor should it start with `I`.

You'll come back shortly to fill in the `reward` method as well.  So far, you are just following the framework we set up in Homework 1.  Later on in this notebook, we'll extend `IobTask0` to `IobTask`, a more efficient subclass that provides a finite-state description of the possible taggings, not just an iterator.

In [ ]:
class IobTask0(TaskSetting):

    Y_alphabet = tuple('IOB')    # the output alphabet for this task
    
    def __init__(self, false_pos_penalty=1):
        super().__init__()
        self.false_pos_penalty = false_pos_penalty    # lambda
        
    def iterate_y(self, *, xx, oo=None, yy_prefix):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    # Could just use the inherited iterate_yy method, but let's extend it with assertions
    # that the strings we're returning are actually legal.  (This might catch a bug, e.g.,
    # you forgot to check whether a character observed in oo was legal given the prefix.)
    def iterate_yy(self, xx, oo=None):
        for yy in super().iterate_yy(xx=xx,oo=oo):
            assert yy[0] != 'I'                # 'I' is illegal following BOS
            assert 'OI' not in ''.join(yy)     # 'I' is illegal following 'O'
            yield yy
                
    def reward(self, *, aa, xx, yy):
        """
        The proxy reward of prediction aa on this sentence if the true chunking is yy.
        """
        # Hint: call reward_F1_triple
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END

    def reward_F1_triple(self, *, aa, xx, yy):
        """
        ***NEW***: returns a triple (true_pos, true, pos) used to compute corpus-level F1: 
           `true_pos` is the number of "true positives" (chunks reported by `aa` that are truly in `yy`) 
           `true` is the number of chunks that are truly in `yy`  
           `pos` is the number of chunks reported by `aa`
        """
        assert len(aa) == len(yy)
        true = sum('B' == y for y in yy)
        pos = sum('B' == a for a in aa)
        true_pos = 0
        I = False    # are we currently inside a true_pos chunk?
        for i in range(len(aa)):
            if I:
                if aa[i] != 'I' or yy[i] != 'I':
                    I = False       # aa and yy didn't both continue the chunk they were both in
                    if aa[i] != 'I' and yy[i] != 'I':
                        true_pos += 1   # aa and yy both ended the chunk they were both in, at same place i-1
            if aa[i] == 'B' and yy[i] == 'B':
                I = True    # both aa and yy started a new chunk at same place i
        true_pos += I       # if I==True, then aa and yy both ended the chunk they were both in at end of string
        return np.array([true, pos, true_pos])

Now let's test that `iterate_yy` does the right thing:

In [ ]:
task0 = IobTask0()

In [ ]:
xx, oo, yy = next(iterate_data('dev-small'))
print(xx,oo)
yys = list(task0.iterate_yy(xx=xx,oo=oo))
(len(yys), yys[:10])    # number of values and the first 10 of them

In [ ]:
# define your own tests here
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


## Rewards for the chunking task

In the `IobTask` class above, we supplied two reward methods — not just the `reward` method that is required by `TaskSetting`, but another method called `reward_F1_triple`.  To see why, first let's review some definitions:
* **True chunks:** `true` is the total number of chunks in the correct $\boldsymbol{y}$ taggings for the test corpus.
* **Positive findings:** `pos` is the total number of chunks in the predicted $\boldsymbol{a}$ taggings for the test corpus.
* **True positives:** `true_pos` is the total number of correctly predicted chunks, i.e., chunks that appear in both $\boldsymbol{y}$ and $\boldsymbol{a}$.  True positives are good!
* **False positives:** `pos - true_pos` is the number of incorrectly predicted chunks.  False positives are bad ("precision errors").
* **False negatives:** `true - true_pos` is the number of missed chunks.  False negatives are bad ("recall errors").
* **True negatives:** This is not usually computed for a chunking task.  True negatives are all the *non*-chunks that we correctly avoid predicting as chunks; there are $O(J^2)$ of them.

It is conventional to evaluate NER and other chunking tasks using the **corpus-level $F_1$ measure**.  Recall that $F_1$ is defined as the harmonic mean of precision and recall, $$F_1 = \left( \frac{\text{precision}^{-1} + \text{recall}^{-1}}{2} \right)^{-1} = \frac{2\cdot\text{true_pos}}{\text{true}+\text{pos}},$$
where $$\text{precision} = \frac{\text{true_pos}}{\text{pos}}, \text{recall} = \frac{\text{true_pos}}{\text{true}}$$
and here `true`, `pos`, and `true_pos` are the *total* counts from the *entire test corpus* (which are unlikely to be 0).  

In the end, we will evaluate our decision agent on test data in this way, with the help of `reward_F1_triple`, which we have defined for you above.  It returns the `(true,pos,true_pos)` triple for the predicted `aa` given the true `yy`.  These triples can be summed over all sentences in the test corpus.  We have defined `IobTask.reward_F1_triple` for you, and here is a method to convert the summed triple to $F_1$:

In [ ]:
def F1(true, pos, true_pos):
    """
    Compute an F1 score from the relevant count triple.
    """
    if true+pos > 0:
        f1 = 2*true_pos / (true+pos)
    else: 
        f1 = 1   # full credit if there were no true chunks and we found none
    precision = (true_pos / pos) if pos > 0 else math.nan  # precision of 0/0 is not well defined
    recall = (true_pos / true) if true > 0 else 1          # recall for 0/0 is 1: full credit if there were no true chunks and we found none
    print(f'F1 {f1}, precision: {precision}, recall: {recall}, true: {true}, pos: {pos}, true_pos: {true_pos}')
    return f1   

In [ ]:
triple = task0.reward_F1_triple(xx=None, 
                                   aa=('O','B','I','B','B','O','B','I','I'), 
                                   yy=('O','B','I','I','B','O','B','I','I'))
triple

In [ ]:
F1(*triple)

But what decision agent should we use?  Unfortunately, it is somewhat tricky (though possible) to construct an efficient Bayes rule for the $F_1$ measure.  
* First of all, the Bayes rule cannot just map each separate sentence to its best tagging.  The "best" tagging of one sentence actually depends on the other sentences!  So it would have to map the entire corpus to the best tagging of the corpus.  This would require annoying changes to our class design or our file formats.

  Mathematically, the problem is that the $F_1$ formula is not simply a sum over sentences $i$.  (Rather, the sums are buried inside the definitions of `true`, `pos`, and `true_pos`.)  As a result, the expected corpus-level $F_1$ is not simply a sum over sentences, so we can't simply maximize it by maximizing the expected $F_1$ for each sentence separately.

  Intuitively, the problem is that to improve the corpus-level $F_1$, it is more helpful to improve whichever is currently lower, precision or recall.  So in deciding what to do on sentence $\boldsymbol{x}_i$, we need to know the precision and recall for other sentences.  If precision is low elsewhere, then we should be cautious and predict only the few chunks of $\boldsymbol{x}_i$ where we are most confident, in order to get high precision even at the expense of recall.  If recall is low elsewhere, then we should be aggressive and predict lots of chunks, in order to get high recall even at the expense of precision.
  
  
* Second, regardless of whether we want to maximize expected $F_1$ for a single sentence or for the whole test corpus, the Bayes rule prediction is tricky to compute efficiently, for similar reasons.  It can't just pick the highest-probability chunks, but must consider the probabilistic dependencies among chunks.  Perhaps surprisingly, the Bayes rule can still be obtained in polynomial time using some interesting probabilities ([Waegemans et al., 2014](http://jmlr.org/papers/volume15/waegeman14a/waegeman14a.pdf)) that we actually could still compute using finite-state dynamic programming — but that is one quantum too complicated for this assignment.  We don't want to fall back on brute force as in homework 1, either, because it would be astronomically slow: we'd have to iterate over all joint chunkings of the entire test corpus.

<a name="proxy"></a>
Thus, as a "proxy" for $F_1$, we will use a different reward measure that is related:
$$R = \text{true_pos} - \lambda \cdot \text{false_pos}$$
This is a linear function and doesn't present the same computational difficulties.  The expected $R$ for the corpus is now just a sum over all the sentences — and it will be relatively easy to use a weighted finite-state machine to compute the expected reward for each sentence.  We will construct a decision agent that maximizes expected $R$ rather than expected $F_1$.

The justification is that $R$ is a fairly reasonable reward measure on its own.  The first term emphasizes recall (encourages positives in hopes of getting true positives).  The second term emphasizes precision (discourages positives for fear of getting false positives).  Thus, by controlling the relative weight of these two terms, $\lambda$ indirectly controls the relative importance of precision and recall in our evaluation.  

We could simply pick a $\lambda$ value and evaluate our chunking system using $R$.  However, it is conventional to evaluate chunking systems using $F_1$ (and maybe for good reason, since $F_1$ strongly penalizes systems with very low precision or very low recall).  Also, it's not clear what $\lambda$ value we should use, whereas $F_1$ explicitly specifies a particular standard tradeoff between precision and recall.  (The use of $F_1$ is meant as an application-independent convention.  If you have a real-world application that calls for a different tradeoff, you could use [$F_\beta$ for some other $\beta$](https://en.wikipedia.org/wiki/F1_score#Formulation).)

So we will in fact evaluate on test data using $F_1$.  To get good test performance, we'll choose $\lambda$ to maximize the $F_1$ measure that our decision agent achieves on dev data.  In other words, although we're using the wrong kind of Bayes rule — a Bayes rule for $R$ rather than $F_1$ — we do have a free hyperparameter in the definition of $R$, and we tune it to trade off precision and recall in the way that $F_1$ likes the best. 

You should fill in the `IobTask0.reward` method above, which computes $R$ given $\boldsymbol{y}$ and $\boldsymbol{a}$.  

Later, we'll construct an FST that can compute `reward` for any $\boldsymbol{y}$ and $\boldsymbol{a}$.  This FST will let you consider all the possible $\boldsymbol{y}$ and $\boldsymbol{a}$ values using dynamic programming rather than brute force.

In [ ]:
# two false positives, then two true positives
assert 0 == task0.reward(xx=None, 
                         aa=('O','B','I','B','B','O','B','I','I'), 
                         yy=('O','B','I','I','B','O','B','I','I'))

In [ ]:
# experiment some more here with `reward_F1_triple`, `F1`, and `reward`
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


The existing method `DecisionAgent.test` calls `reward`, so it will compute the *proxy* reward on the test set.  Let's patch the `DecisionAgent` class to add a new method, so that all decision agents will also know how to compute $F_1$ on the test set.  This patch will be inherited by all subclasses and instances of `DecisionAgent`.

In [ ]:
def test_F1(self, dataset):
    """
    Run the decision rule on all the examples in `dataset` and return the F1 score.
    """
    counts = np.zeros(3)  # return true, pos, true_pos
    for c, (xx, oo, yy) in enumerate(dataset):
        aa = self.decision(xx=xx, oo=oo)
        counts += self.task.reward_F1_triple(aa=aa, xx=xx, yy=yy)   # running total
        if c % 50 == 49: sys.stdout.write('\r\tevaluated reward on {} examples'.format(c+1))
    sys.stdout.write('\n')

    # compute the f1 score.  We use a rearranged form of the formula, to reduce the chances of division by 0.
    true, pos, true_pos = counts
    
    return F1(true, pos, true_pos)

DecisionAgent.test_F1 = test_F1    # patch DecisionAgent to add a new method

# Part 2 — Modeling with a Conditional Random Field

**Model / Features:**
<img src="images/crf.png" width=750px ></img>

Our model is a CRF: $$p(\boldsymbol{y} \mid \boldsymbol{x}) = \frac{1}{Z(\boldsymbol{x})} \exp G(\boldsymbol{x},\boldsymbol{y}) = \frac{1}{Z(\boldsymbol{x})} \left(\exp \boldsymbol{\theta}\cdot\text{features}(\boldsymbol{x},\boldsymbol{y})\right)$$ 
Each of our features is a **count feature** that returns the count of some particular configuration in $(\boldsymbol{x},\boldsymbol{y})$.  Thus, $G(\boldsymbol{x},\boldsymbol{y})$ is the total weight according to $\boldsymbol{\theta}$ of all the configurations that appear in $(\boldsymbol{x},\boldsymbol{y})$.

We use the following features:
* Tag-tag features.  For example, the `I_O` feature fires on every `I O` bigram (which rewards or punishes each named entity that ends in the middle of a sentence).  Exactly one tag-tag feature fires at each position $j$, which adds a term $g_{tt}(y_j,y_{j+1})$ into the score, or equivalently multiplies a factor $\psi_{tt}(y_j,y_{j+1}) = \exp g_{tt}(y_j,y_{j+1})$ into the unnormalized probability, as shown in the figure.
    * Tag-tag features also fire at the boundaries of $\boldsymbol{y}$.  For example, the `I_EOS` feature fires if there is an `I EOS` bigram (which rewards or punishes any named entity that ends at the end of a sentence).  The factors contributed by these two features are labeled in the figure as $\psi_{\text{BOS}}$ and $\psi_{\text{EOS}}$.
* Tag-word features.  For example, the `B_brussel` feature fires for every token of the word `brussel` that starts a chunk.  Exactly one tag-word feature fires at each position $j$, which adds a term $g_{tw}$ into the score, or equivalently multiplies a factor $\psi_{tw} = \exp g_{tw}$ into the unnormalized probability, as shown in the figure.

The above factors are analogous to the transition and emission probabilities of HMMs.  This feature set deliberately limits the dependencies among the tags $y_j$ to be bigram dependencies.  This ensures that there exists a simple "trellis" WFSA that can score every possible $\boldsymbol{y}$ — exactly the same WFSA as for an HMM (with about $J\cdot|\mathcal{Y}|$ states and $J\cdot|\mathcal{Y}|^2$ arcs).  Inference will take time proportional to the size of the trellis.

Recall that the features in a CRF model $p(\boldsymbol{y} \mid \boldsymbol{x})$ can depend freely on any property of $\boldsymbol{x}$.  This does not change the topology of the trellis, but only makes its the weights dependent on $\boldsymbol{x}$.  To illustrate some of this generality, we also include:
* Tag-previous-word features.  For example, the `B_brussel_-1` feature fires for every chunk that is immediately preceded by the word `brussel`.  At each position, the factor contributed by these features is labeled as $\psi_{tw_{-1}}$.  Note that the previous word may be $\text{BOS}$.

In short, we have $$p(\boldsymbol{y} \mid \boldsymbol{x}) = \frac{1}{Z(\boldsymbol{x})} \left(\prod_{j=0}^{J} \psi_{tt}(y_j,y_{j+1})\right) \left(\prod_{j=1}^J \psi_{tw}(y_j,x_j) \psi_{tw_{-1}}(y_j,x_{j-1})\right)$$

Many other features could be added in principle.  Obviously, we could use not just $\psi_{tw_{-1}}$ but also $\psi_{tw_{-2}}, \psi_{tw_{+1}}, \psi_{tw_{+2}}$.  We could use features that relate tag $y_j$ to the spelling of word $x_j$ and to its position $j$ in the sentence.  

(Remark: A more advanced move would be to have features that examine entire chunks, not just tag unigrams and bigrams.  For example, a feature might fire on every span $\boldsymbol{x}_{i:j}$ that appears in a list of university names and is marked as a chunk by $\boldsymbol{y}$.  Such a "segmental CRF" goes beyond the bigram features shown above, and in general would require us to increase the trellis size and the runtime of inference from $O(J)$ to $O(J^2)$.)

## Implementing the model

As the formula makes clear, a CRF is just a Boltzmann distribution, expressed as a product of factors $\psi$ that refer to different (possibly overlapping) portions of the output structure.  Thus, our implementation will extend the `BoltzmannModel` class from homework 1.  (This means we are still using brute-force algorithms — we'll fix that later.)

We'll define our new subclass in such a way that it could be used for *any* tagging task — not just IOB tagging, but also part-of-speech tagging or syllable stress tagging.  There is a clean separation of concerns between the task setting above and the probability model below:
* The choice of the IOB alphabet, the restriction on legal IOB tag sequences, and the special meanings of the IOB tags show up only in the task setting and its reward function.
* Conversely, the bigram CRF structure and features show up only in the probability model. 

So it's possible to mix-and-match different tagging tasks with different tagging models.

Again, we'll start with a brute-force model.  So just fill in the implementation of `score_with_gradient` below, remembering that the gradient $\nabla G_{\boldsymbol{\theta}}$ is just the vector of feature counts.  You will also have to fill in `initialize_params` to set $\boldsymbol{\theta} = 0$.  You should have $|\boldsymbol{\mathcal{Y}}|^2 + 2 \cdot |\boldsymbol{\mathcal{Y}}| + 2 \cdot |\boldsymbol{\mathcal{X}}| \cdot |\boldsymbol{\mathcal{Y}}|$ parameters in your model, one for each of the features described above.

#### Managing the parameters

You should use an attribute called `.params`, as in homework 1.  (Homework 1 actually made it a property, but an ordinary attribute is simpler and will work better with some of the designs below.  As mentioned above, we changed `Probability.params` accordingly in `seq2class_homework1.py`, and so should you.)  

But what kind of object is the value of `.params`?  How you store the parameters is up to you.  Some options:
* You could stick with a single parameter vector $\theta$ (exactly like `LogLinearModel` in homework 1).
    * You could then decide to store the tag-tag parameters in one part of $\theta$ and the tag-word parameters in another part.  By integerizing the tags and words, and using a systematic layout, you can use simple arithmetic to compute the distinct integer index for each parameter.  This is essentially a customized integerizer for the parameter names.
    * Alternatively, instead of calculating indices via arithmetic, you could use a generic `Integerizer` object to map entire parameter names such as (tag, word) tuples to distinct integers that index the parameter vector.  This is easier, more elegant, probably less error-prone, and more reusable, if a bit slower.  
        * *Caution:* When naming your parameters, be careful not to confuse the word 'I' with the tag 'I'.  In the same spirit, the boundary markers should not be represented as strings `BOS` and `EOS`, to prevent confusion if those strings also serve as tags or words.  (A good choice is to instead use the Python object `None` for any tag or word that is beyond the edge of the string, e.g., `xx[0]==None`, `yy[0]==None`, `yy[len(yy)]==None`.)
* Perhaps more elegantly, instead of forcing the parameters into a vector shape, you could use distinct parameter *matrices* for the tag-tag feature weights and the tag-word feature weights.  Then the `params` object should store these matrices.  You will need a new class for this (call it `BigramCrfParams`).
    * The gradient has the same shape as the parameter vector, so gradient would be stored in an object of the same class.  This class should implement at least `__isub__`  and `__rmul__` [methods](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types), which will be called when `SGDTrainer` invokes `-=` and `*` to subtract a multiple of the gradient from the current `params`.  See `paramdict.py` for an illustration of how to do this.
* In fact, as long as `params` is a new kind of object, that object doesn't even have to contain matrices at all.  It could just be a generic `ParamDict` (defined above) that maps parameter names directly to their weights.  `ParamDict` is not specific to our CRF, but could be used for any model, since it is very flexible: unlike `BigramCrfParams`, it doesn't commit to using a particular collection of matrices.  Just as with the `Integerizer` solution above, this elegant generic solution is easier than the customized solution but may be a bit slower.

#### Sparsity

If you wish to be asymptotically efficient, make use of *sparse* vectors or matrices.  The CRF's *parameters* can be naturally and efficiently stored in dense vectors or matrices.  However, the SGD *gradient* on a single training example $(\boldsymbol{x},\boldsymbol{y})$ will be sparse, because only a few feature weights affect $p_\theta(\boldsymbol{y}\mid\boldsymbol{x})$; the gradient with respect to the other parameters is therefore 0.  

Exploiting sparsity means that you shouldn't waste any time on all those zeroes.  You want the runtime for each training example to be proportional to the number of features that actually fire on the example, not the total number of features.  For example, an SGD step on a 20-word training sentence should adjust the tag-word parameters for only the ≤ 20 distinct words in that sentence, which might be a tiny subset of the full parameter vector if the vocabulary is large.  Using a sparse gradient vector is a way to avoid storing and adding useless zeroes for the rest of the parameters.  

Here are some options if you want to try a sparse solution.  Be warned that in Python, these solutions *might* be slower than using dense numpy arrays.  The dense numpy array operations are highly optimized: they make low-level library calls that exploit hardware parallelism (vector processing on the CPU).  The sparse solutions are *asymptotically* faster, but they have to run explicit loops, which has overhead, especially in an interpreted language like Python.

* The `ParamDict` solution is naturally sparse.
* You could try SciPy's implementation of sparse vectors and matrices, `scipy.sparse.dok_matrix`.
* Here's a fast data structure that mixes dense and sparse.  (This is what Jason would probably do in a streamlined C++ implementation.)  Store the params and gradient in parallel arrays — which allows rapid random access to the integerized parameters and their partial derivatives — but also pair the gradient array with a list of the indices of its (few) nonzero elements.  Thanks to this list, you can make each SGD step take time proportional to the number of features used on this step:
    * To avoid creating a new gradient array (with lots of zeros) for each training example, `BigramCrfModel` should just reuse the same gradient array for every call to `score_with_gradient`.
    * Taking an SGD step only requires visiting the nonzero elements.  
    * These elements can be reset to zero at the start of the next call of `score_with_gradient`.  
        * Better yet, you can write a specialized method that handles `params -= stepsize * gradient; gradient = 0` all at once.  This method iterates only once through the nonzero elements of gradient, applying them to params and resetting them to 0 as they are applied.  This avoids having to iterate again later to reset them.  It also means that visiting the same element a second time will have no additional effect (since it's been reset to 0), which means that you can add the index to the list without checking for duplicates.  (You can prevent most duplicates cheaply by declining to add an index when you update a non-0 element, since that element will already be on the list.  However, a duplicate will still arise in the rare case of updating from 0 to non-0 back to 0 to non-0; so it's important to know that this will not introduce an error.)
    * To fit with the rest of the object-oriented design, you should create a simple class to represent the gradient vector as an (array, list) pair.

#### The vocabulary of word types

Because the model uses tag-word and tag-previous-word features, the number of features depends on the size of the vocabulary.  Some of the strategies above need to know the size of the vocabulary in advance.  Thus, your constructor for `BigramCrfModel` should take the vocabulary as an argument.  This can be any collection of word types or tokens (generally derived from training data).  The constructor will probably want to turn it into a `Set` or an `Integerizer`.  If word $w_i$ is not in the vocabulary (OOV), then no tag-word feature will fire at position $i$. 

The `ParamDict` strategy has the nice property that you *don't* actually need to know the full set of parameters in advance.  New parameters will start out as 0 when first accessed, and will be added lazily to the dictionary when first modified.  In fact, this would make it easy to implement an infinite-vocabulary model, where the `ParamDict` only stores nonzero weights for the finitely many words observed in training so far.  For this homework, however, you should respect the `vocab` argument to the constructor, which defines a finite vocabulary.  So you should explicitly avoid computing features on OOV words, although it is still ok to add the in-vocabulary features lazily to the `ParamDict` as you encounter them.

Complete the `BigramCrfModel0` class below, still following the framework of Homework 1.  Later on in this notebook, we'll extend it to `IobTask`, a more efficient subclass that provides a finite-state description of the probability distribution, in order to allow dynamic programming.

In [ ]:
class BigramCrfModel0(BoltzmannModel):
    """
    A 1st-order CRF model with a particular feature set described in the assignment.
    The tag types are given by task.Y_alphabet.
    The word types are supplied to the constructor via an iterable collection of
    word types or tokens.
    """
    def __init__(self, task, *, vocab):
        super().__init__(task)   # among other things, this calls `initialize_params`
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END

    def initialize_params(self):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def score_with_gradient(self, *, xx, yy):
        """
        Compute the feature vector given `xx` and `yy` sequences.
        Then return the score and its gradient (i.e., the feature vector).
        """
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    

In [ ]:
# A little example to check that the right features are extracted.
# Since we're using the initial parameter vector of 0, the score will be 0, 
# but the gradient will show the feature counts, which you can check manually.
model0 = BigramCrfModel0(task0,vocab=['foo','bar'])   
model0.score_with_gradient(yy=('B','I','I','O'),xx=('bar','bar','oov','foo'))

## Trying out the model using brute-force methods

This is enough to get our methods from Homework 1 running.  They are slow brute-force methods, so let's train on `train-small` and evaluate the performance on `dev-small`.  We do have to add a way of testing using $F_1$ reward.

In [ ]:
# Define the vocabulary.
import itertools
def vocab_from_data(examples):
    """
    Extract a vocabulary from the dataset.  
    Currently we use all words that appear at least once in the dataset.
    """
    # Iterate through all words in all input sentences (xx), and construct a set. 
    return set(itertools.chain.from_iterable(map((lambda e: e.xx), examples)))  

data_small = list(iterate_data('train-small'))
vocab_small = vocab_from_data(data_small)
(len(vocab_small), list(islice(vocab_small,10)))  # size of vocab and 10 words from it

In [ ]:
class TrainableBigramCrfModel0(L2LogLikelihood, BigramCrfModel0):   
    pass   # inherit everything from the two parents

model0 = TrainableBigramCrfModel0(task0, 
                                  vocab=vocab_small, 
                                  regularization_coeff=15,
                                  num_examples=len(data_small))

sgd_trainer = SGDTrainer(epochs=10)
%time sgd_trainer.train(model0, data_small)

Check our F1 score on short sentence development data set.  Decoding is performed using the brute-force decision agent, which iterates through the entire domain of each $\mathcal Y_x$.

In [ ]:
agent0 = ViterbiAgent(task0, model0)
agent0.test_F1(iterate_data('dev-small'))

## Part 3 — Finite-state methods

We are now going to review FSTs, and go through a small tutorial of working with FSTs in Python.  Our eventual goal is to use efficient FST operations to implement a Bayes rule for the chunking task under our CRF model.

#### If you have not already done so, install our OpenFST Python library:
   ```bash
   source ~/path/to/anaconda3/bin/activate  # activate our anaconda environment
   which python  # should print /home/USERNAME/path/to/anaconda3/bin/python
   pip install --upgrade 'git+https://github.com/matthewfl/openfst-wrapper.git'
   # ^^ this will take about 5-10 minutes (tested on ugrad machines)
   ```

Throughout this assignment, you may find it helpful to reference OpenFST's available operations **[here](http://www.openfst.org/twiki/bin/view/FST/FstQuickTour#Available_FST_Operations)**.

Additionally, you can find the documentation on mfst, our FST library, included in its source **[here](https://github.com/matthewfl/openfst-wrapper/blob/master/mfst/__init__.py)**.
  
Note that OpenFST and mfst refer to all finite-state machines as FSTs.
However, in this notebook, we will use FSA to refer to the special case of a finite-state *acceptor*,
and reserve FST for finite-state *transducers* (whose inputs and outputs may be different).  This serves as some documentation of the intended types.
  
  
<span style='color:red;font-size: 20px'>Warning:</span> OpenFST has some of the <u>*worst*</u> error handling.  If you do something wrong, there is a decent chance that you are just going to get the garbage as the output.  You should attempt to sanity check any operations with OpenFST using small examples.  Some functions will fail to terminate if you give them the wrong input.  You will have to check the OpenFST documentation to determine if there are any restrictions on what a function can handle or if the runtime of a method will be *acceptable*.  OpenFST provides `FST().verify` which you might find helpful in performing basic sanity checks when debugging your FSTs.

In [ ]:
try:
    from mfst import ( FST, AbstractSemiringWeight, PythonValueSemiringWeight, 
                       RealSemiringWeight, BooleanSemiringWeight, MaxPlusSemiringWeight )
except ImportError:
    print('Error: Need to install mfst package; see instructions in notebook')


As a simple example, here's a simple FST with two states, `start` and `end`.  There is a single arc between these two states with a label of `i:o/1`.  This arc can be traversed on input character `i`, which results in emitting output character `o` and incurring a weight of `1`.

OpenFST internally uses integers as its input and output symbols, with `0` representing ε (the empty string `''`).  However, the mfst package will also let you specify characters such as `i` or `θ`; a character will be automatically [converted](https://docs.python.org/3/library/functions.html#ord) to an integer, namely its Unicode code point.  (If you specify any characters when creating the FST, the integer labels will be [converted back](https://docs.python.org/3/library/functions.html#chr) to characters when the FST is drawn.) 
If you want your input and output symbols to come from some other set of objects, such as words, then you should convert them to integers yourself, using an `Integerizer` that has been initialized with `''` as its first object.

In [ ]:
myfst = FST()                    # new FST
start = myfst.add_state()        # add a state (represented as an integer)
myfst.initial_state = start      # designate it as the initial state
end = myfst.add_state()          # add another state
myfst.set_final_weight(end, 1)   # designate it as a final state, with stopping weight of 1
myfst.add_arc(start, end, input_label='i', output_label='o', weight=1)   # add an edge labeled i:o/1

# look at the FST
myfst

The initial state is shown in green, and final states (other than the initial state) are shown in red.

The default weight for both `set_final_weight` and `add_arc`is `1` (in general, the semiring's ① value) , so the above calls could actually have been simplified to omit the weight argument.  As you can see, **default weights are not shown in the drawing**.

To check your understanding, create a cycle by adding an edge from the final state back to the initial state, labeled with `ε:z/2`.  Remember that ε is represented by the integer label 0.

In [ ]:
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END
myfst

You can use some of [mfst](https://github.com/matthewfl/openfst-wrapper/blob/master/mfst/__init__.py)'s methods to inspect your FST:

In [ ]:
(start,end)  # states are represented by integers, as in the drawing

In [ ]:
myfst.get_final_weight(start)   # start is not a final state in myfst

In [ ]:
myfst.get_arcs(start)  # the arcs leaving start

In [ ]:
myarc = myfst.get_arcs(start)[0]
(chr(myarc.input_label), chr(myarc.output_label), myarc.weight.value)

In [ ]:
paths = myfst.iterate_paths()    # iterator over all paths
list(islice(paths,10))           # first 10 paths out of infinitely many

Recall that an FSA that recognizes a (regular) language $L$ is traditionally implemented as an FST that transduces any string in $L$ to itself (with weight one in the semiring), and rejects any other string in $L$.  

As a special convenience, here's a way to construct an unweighted FSA that accepts exactly one string, assigning it the `BooleanSemiringWeight` of `True`.

In [ ]:
FST('hello')

Usually it's fine to use an unweighted FSA, since `mfst` kindly permits you to compose these nicely with machines in other semirings.  However, if you wanted to create a single-string FST in some other semiring, you can do it like this:

In [ ]:
base_fst = FST(RealSemiringWeight)
base_fst.create_from_string('hello')

Or like this, for an integer sequence:

In [ ]:
base_fst.create_from_string( (1234, 1337) )

Recall that [compose](http://www.openfst.org/twiki/bin/view/FST/ComposeDoc) is the way to feed a string into an FST.  In general, it takes all the output strings of one FST and feeds them as input to a second FST.
If no output of the first FST is accepted by the second FST, then we will get an empty FST as the result.

In [ ]:
FST('hello').compose(myfst)  # hello is not in the domain of myfst

In [ ]:
FST('i').compose(myfst)

Using `myfst` from above, what input string will generate the output `ozozo`?

In [ ]:
### STUDENTS START
### input_string = FILL IN
raise NotImplementedError()  # REPLACE ME
### STUDENTS END
FST(input_string).compose(myfst).get_unique_output_string()

### Paths through FSTs

As we will see below, with the right choice of semiring, OpenFST's [`shortest_path`](http://www.openfst.org/twiki/bin/view/FST/ShortestPathDoc) method will run the Viterbi algorithm for finding the best path.

First, here is [the hot-cold lattice from the NLP class](https://www.cs.jhu.edu/~jason/papers/#eisner-2002-tnlp).

In [ ]:
def build_HC_fsa(fsa, n=3, same_weight=1, change_weight=1):
    """
    Build a weighted FSA on weather sequences.  
    The first argument should be an existing FSA to which we will add states.
    It should be designated as an acceptor, in which case we don't have to specify 
    `output_label` on the arcs (it will automatically be set equal to `input_label`).
    """
    start = fsa.add_state()
    fsa.initial_state = start
    h = [fsa.add_state() for i in range(n)]
    c = [fsa.add_state() for i in range(n)]
    fsa.add_arc(start, h[0], input_label='H')  # default weight of semiring one
    fsa.add_arc(start, c[0], input_label='C')
    for i in range(n - 1):
        fsa.add_arc(h[i], h[i+1], input_label='H', weight=same_weight)
        fsa.add_arc(c[i], c[i+1], input_label='C', weight=same_weight)
        fsa.add_arc(h[i], c[i+1], input_label='C', weight=change_weight)
        fsa.add_arc(c[i], h[i+1], input_label='H', weight=change_weight)
    fsa.set_final_weight(c[-1])
    fsa.set_final_weight(h[-1])

weather = FST(acceptor=True)
build_HC_fsa(weather, same_weight=2, change_weight=5)
weather

### The Tropical semiring

Weights in the above FSA are simply arbitrary Python objects equipped with their built-in `+` and `*` operators (that is, `__add__` and `__mul__` methods).  This is mfst's default semiring, `PythonValueSemiringWeight`.  

But if we want to find the best path, we need to instead use the $(\min,+)$ semiring.  This is commonly called the "tropical semiring" (in homage to Brazilian researcher Imre Simon).

To review the semiring, fill in the table below.  This is technically speaking not just a semiring but a "division semiring" -- the left-division operation included below is needed by the determinization and minimization algorithms for weighted FSAs and FSTs.  The $\le$ [operation](http://www.openfst.org/twiki/bin/view/FST/FstAdvancedUsage#Natural_Orders) is defined in terms of $\oplus$ (see below); in this case it happens to be a total order, so the weight of the shortest path is well-defined, and the semiring is said to have the [path property](http://www.openfst.org/twiki/bin/view/FST/FstWeightRequirements).

$$
\begin{array}{|c|c|c|c|} \hline
\text{Semiring operation} & \text{Symbol} & \text{Axioms} & \text{Definition in tropical semiring} \\ \hline
\text{Zero} & ⓪ & ⓪ \oplus a = a  & \infty \\
& & ⓪\otimes a = a \otimes ⓪ = ⓪ \\ \hline
\text{One} & ① & ① \otimes a = a \otimes ① = a & \color{red}{\text{FILL IN}} \\ \hline
\text{Add} & \oplus & \text{associative, commutative} & c = \text{min}(a, b) \\ \hline
\text{Multiply} & \otimes & \text{associative, distributive over $\oplus$} & \color{red}{\text{FILL IN}} \\ \hline
\text{Left-divide} & {}^{-1} & c = a^{-1}b \Rightarrow a \otimes c = b& \color{red}{\text{FILL IN}} \\ \hline
\text{Compare} & \leq & a \le b \iff a \oplus b = a & \color{red}{\text{FILL IN}} \\ \hline
\end{array}
$$

To check your understanding, what would change in these definitions if you were looking for the path of *maximum* total weight instead of *minimum* total weight? 

**Answer**: <span style='color:red'>FILL IN</span>

Now let's implement the semiring in Python, so you can see how that's done.  (See [semirings.py](https://github.com/matthewfl/openfst-wrapper/blob/master/mfst/__init__.py) for more examples.)

In [ ]:
class TropicalSemiring(RealSemiringWeight):
    
    # tell OpenFST that <= is a total order (the "path property")
    semiring_properties = 'path'  
    
    # let self.value be the path length of this edge
    
    def __init__(self, v):
        super().__init__(v)  # sets self.value = v
            
    def __add__(self, other):
        # return a new instance of TropicalSemiring with value: self (+) other
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    
    def __mul__(self, other):
        # self (*) other
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
    
    def __div__(self, other):
        # self (/) other
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        
    def __eq__(self, other):
        return isinstance(other, TropicalSemiring) and self.value == other.value
    
    def __hash__(self):  # __hash__ is required if defining __eq__
        return hash(self.value)
    
TropicalSemiring.zero = TropicalSemiring(math.inf)
# TropicalSemiring.one = FILL IN
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END


In [ ]:
def test_semiring(semiring):
    # some basic sanity checks for the semiring
    one = semiring.one
    zero = semiring.zero 
    two = one + one
    assert one * zero == zero
    assert two * zero == zero
    assert one + zero == one
    assert two + zero == two
    assert two / two == one
    
test_semiring(TropicalSemiring)

In [ ]:
# check TropicalSemiring operations
(
    TropicalSemiring.zero + TropicalSemiring.one, 
    TropicalSemiring.zero * TropicalSemiring.one, 
    TropicalSemiring(1),                        # not equal to TropicalSemiring.one
    TropicalSemiring(6) / TropicalSemiring(2)   # not equal to TropicalSemiring(6/2)
)

Now let's re-build the weather FSA using `TropicalSemiring` weights.

In [ ]:
weather = FST(TropicalSemiring, acceptor=True)
build_HC_fsa(weather, same_weight=2, change_weight=5)
print(weather.semiring)
print(weather)
weather

Again, you can extract all paths labeled with a particular input string by composing that string with the FSA or FST.  The weight of a path is the product of the weights along the path.

In [ ]:
weather.compose(FST('HCC'))

### More finite-state operations

[**Weight pushing**](http://www.openfst.org/twiki/bin/view/FST/PushDoc) redistributes the weights along the paths.  Every path still has the same weight as before (defined by $\otimes$).  However, pushing the weights toward the initial state means that we will "take the pain" as early as possible on the path.  Let's see how it works on the hot-cold FSA:

In [ ]:
weather.push()

How does pushing work?  In the tropical semiring, suppose the paths *from* a certain state have total weights of 13, 17, 23, and 27.  Reaching that state means that we will incur a future cost of *at least* 13.  The `push` operation will push that guaranteed cost of 13 "backwards through the state" so that it instead appears on the paths leading *to* the state.  Now the paths from the state have total weights of only 0, 4, 10, and 14.  

Weights are recursively pushed back as far as possible.  Let $\beta(t)$ denote the total weight of all paths from state $t$ — where "total" uses $\oplus$, so it is the minimum (e.g., 13) in the case of the tropical semiring.  Formally, the pushing algorithm replaces the weight $w$ on an arc $s \stackrel{w}{\rightarrow} t$ with $\beta(s)^{-1}w \otimes \beta(t)$.  Thus, the $w$ arc receives an extra weight of $\beta(t)$ that has been pushed back through $t$ from all the paths leaving $t$, but then the arc (like all of $s$'s out-arcs) pushes a weight of $\beta(s)$ back through $s$.

The above paragraph handles arc weights but not state weights.  Figure out and explain how the pushing algorithm should handle the weights at initial and final states, where the weight of a path from initial state $a$ to final state $z$ is $\text{initial_weight}(a) \otimes \text{arc_weight} \otimes \cdots \otimes \text{arc_weight} \otimes \text{final_weight}(z)$.  Convince yourself that after this algorithm has been run, all path weights are preserved and we have $\beta(s) = ①$ at all states $s$.  Note that $\beta(s)$ sums over "suffix paths" from $t$: this includes the initial-state weight 

<b>Answer:</b> <span style='color:red'>FILL IN</span>

You can test your understanding in the notebook below this cell.  Be warned, however, that OpenFST doesn't actually support general initial-state weights.  Rather, it requires a single dedicated initial state whose initial weight is ①, and all other states have initial weight of ⓪.   This involves no loss of generality, since any FST with initial-state weights is equivalent to one in OpenFST's restricted form.  How does this restriction change the pushing construction?

<b>Answer:</b> <span style='color:red'>FILL IN</span>

Most important, you want the weight of the shortest path in an FSA or FST.  Where can you find that in general after you invoke weight pushing?  Check your answer using the above drawing of `hc_fsa.push()`.

<b>Answer:</b> <span style='color:red'>FILL IN</span>

Pushing is called as a subroutine in weighted FSA minimization, because two states $s,s'$ that were not originally equivalent (mergeable) can become equivalent after pushing, if they accepted the same weighted suffix language up to a scalar multiplier.  When our FSA is minimized as below, which states are merged?  Were they already equivalent before pushing?  

<b>Answer:</b> <span style='color:red'>FILL IN</span>

In [ ]:
weather.minimize()

What is different if `same_weight` and `change_weight` are equal, as below?  Why?

<b>Answer:</b> <span style='color:red'>FILL IN</span>

In [ ]:
weather2 = FST(TropicalSemiring, acceptor=True)
build_HC_fsa(weather2, same_weight=5, change_weight=5)
weather2.minimize()   # you may also want to look at the non-minimized version

Recall we sometimes have to deal with strings like: `oo='H?H'`, which is compatible with both `HCH` and `HHH`.  For this, you'll need something a little more general than `fst.create_from_string`.  

In [ ]:
def fsa_from_observable(self, oo, alphabet, wildcard='?'):
    """
    Return an FSA that accepts all sequences compatible with `oo`.  The `wildcard`
    symbol in `oo` is allowed to match any element of `alphabet`.  The FSA
    uses the same semiring and other parameters as `self`.  
    """
    fsa = self.constructor(acceptor=True)
    ### STUDENTS START
    raise NotImplementedError()  # REPLACE ME
    ### STUDENTS END
    return fsa

FST.create_from_observable = fsa_from_observable    # patch the FST class to add this method

In [ ]:
fsa_oo = FST(BooleanSemiringWeight).create_from_observable('H?H', 'HC')
fsa_oo.verify()   # check that the FSA is well-formed
fsa_oo

In [ ]:
posterior = weather.compose(fsa_oo)      # restrict `yy` values to those compatible with observations `oo`
posterior

In [ ]:
print(list(posterior.states))
print(posterior.shortest_distance())              # alpha values (total semiring weight of all prefix paths TO each state)
print(posterior.shortest_distance(reverse=True))  # beta values (total semiring weight of all suffix paths FROM each state)

We can run the Viterbi algorithm on the `posterior` FSA to extract the lowest-weight path given our model `weather` and observations `oo`.  This turns out to be labeled `HHH`.  Why does this answer make sense?  Can you guess why OpenFST did not use 0 as the number of the initial state of `fsa_best_yy`?

<b>Answer:</b> <span style='color:red'>FILL IN</span>

In [ ]:
# 1-best posterior guess of weather given the observation fsa_oo
fsa_best_yy = posterior.shortest_path()   
display(fsa_best_yy)
print(next(fsa_best_yy.iterate_paths()))
best_yy = fsa_best_yy.get_unique_output_string()
best_yy

### The real semiring
<a name="ExpectationSemiring"></a>

Finite-state machines are good at defining certain kinds of probability distributions.  We are interested in the distribution over an FSA's paths $\boldsymbol{y} = y_1 y_2 \cdots$, where the $y$ symbols denote edges and a final state.  A training data observation $\boldsymbol{o}$ is in general a regular language that is compatible with multiple possible paths.  We say that $\boldsymbol{o}$ is a "fully supervised" training example when it completely identifies the path.  This is the case when it specifies the path's complete label string (with no wildcards `?`) and the FSA is deterministic (as will be true for our CRF model).

To define the unnormalized probabilities of a Boltzmann distribution over the FSA paths, you can weight your FSA in the real semring $(\oplus,\otimes) = (+,\times)$.  The weight of a path $\boldsymbol{y}$ should be its unnormalized probability $\tilde{p}(\boldsymbol{y})$.  Your job is to associate a probability factor $\psi = \exp g$ with each edge, so that a path's unnormalized probability $\tilde{p}(\boldsymbol{y}) = \exp G(\boldsymbol{y}) = \exp (g(y_1) + g(y_2) + \cdots) = \psi(y_1) \cdot \psi(y_2) \cdot \cdots$ is given by the *product* of the factors on its edges and its final state.  The path's normalized probability is then $\tilde{p}(\boldsymbol{y})/Z$, where $Z$ is the total weight of all paths, which can be computed efficiently via the backward algorithm as $\beta(\text{initial state})$.  

In [ ]:
# We were using negative log probabilities in the tropical semiring (2 and 5)
# Now let's convert those to probabilities in the real semiring (to 3 decimal digits)
weather = FST(RealSemiringWeight, acceptor=True)  
build_HC_fsa(weather, same_weight=round(math.exp(-2),3), change_weight=round(math.exp(-5),3))
weather

Recall that the weight pushing operation tries to ensure that the out-arcs from a node sum to ①, which in this case means that they sum to 1.  Thus, it converts this globally normalized FSA to a locally normalized one!

In [ ]:
weather.sum_paths()   # computes Z

In [ ]:
weather.push()

Once the machine is locally normalized, you can sample paths by working left to right from the initial state.

In [ ]:
weather.push().random_path(8)

By determinizing a large sample, you can see the number of times each of the 8 possibilities was chosen.  Are there any that didn't show up in the sample?  Why?  (You may want to run this more than once.)

**Answer**: <span style='color:red'>FILL IN</span>

In [ ]:
weather.push().random_path(200).determinize()

### The expectation semiring

The expectation semiring was introduced by [Eisner (2002)](https://www.cs.jhu.edu/~jason/papers/#eisner-2002-acl-fst) (and further studied by [Li and Eisner (2009)](https://www.cs.jhu.edu/~jason/papers/#li-eisner-2009)) as a way to define not only a distribution as above, but also the expected reward (of a given plan) under that distribution. It lets you associate not only a factor $\psi$ but also a reward $r$ with each edge, so that the path's reward $R(\boldsymbol{y}) = r(y_1) = r(y_2) + \cdots$ is the *sum* of rewards on its edges and its final state.

We'll arrange for the weight of a single path $\boldsymbol{y}$ to be the pair $\langle \tilde{p}(\boldsymbol{y}), \tilde{p}(\boldsymbol{y}) R(\boldsymbol{y}) \rangle$.
We must define $\oplus$ and $\otimes$ operations on such pairs such that:
* $\oplus$: The total weight of all paths $\beta(\text{initial state})$ is the pair $\langle \sum_{\boldsymbol{y}} \tilde{p}(\boldsymbol{y}), \sum_{\boldsymbol{y}} \tilde{p}(\boldsymbol{y}) R(\boldsymbol{y}) \rangle$.  This allows us to get the expected reward by dividing the pair's second element by its first element: $$\frac{\sum_{\boldsymbol{y}} \tilde{p}(\boldsymbol{y}) R(\boldsymbol{y})}{\sum_{\boldsymbol{y}} \tilde{p}(\boldsymbol{y})} = \sum_{\boldsymbol{y}} \frac{\tilde{p}(\boldsymbol{y})}{Z} R(\boldsymbol{y}) = \sum_{\boldsymbol{y}} p(\boldsymbol{y}) R(\boldsymbol{y}) = \mathbb{E}_{\boldsymbol{y} \sim p}[R(\boldsymbol{y})]$$
* $\otimes$: The weight of a single path $\langle \tilde{p}(\boldsymbol{y}), \tilde{p}(\boldsymbol{y}) R(\boldsymbol{y}) \rangle$ can be obtained as the product of weights $\langle \psi(y_i), \psi(y_i) r(y_i) \rangle$ associated with its edges and final state.

This is accomplished by the definitions below.  Notice that if you ignore the second element of each pair, you have the real semiring.
$$
\begin{array}{|c|c|c|c|} \hline
\text{Semiring operation} & \text{Symbol} & \text{Definition in expectation semiring} \\ \hline
\text{Zero} & ⓪ & \langle 0, 0 \rangle \\ \hline
\text{One} & ① & \langle 1, 0 \rangle \\ \hline
\text{Add} & \langle \tilde{p}_1, \tilde{r}_1 \rangle \oplus \langle \tilde{p}_2, \tilde{r}_2 \rangle & \langle \tilde{p}_1+\tilde{p}_2, \tilde{r}_1+\tilde{r}_2 \rangle \\ \hline
\text{Multiply} & \langle \tilde{p}_1, \tilde{r}_1 \rangle \oplus \langle \tilde{p}_2, \tilde{r}_2 \rangle & \langle \tilde{p}_1 \tilde{p}_2, \tilde{p_1} \tilde{r}_2+\tilde{r}_1 \tilde{p}_2 \rangle  \\ \hline
\text{Left-divide} & \langle \tilde{p},\tilde{r} \rangle^{-1} & \color{red}{\text{FILL IN}} \\ \hline
\end{array}
$$

In general, a weight in the expectation semiring is a pair $\langle \tilde{p}, \tilde{r} \rangle$ that summarizes a set of paths (or partial paths, including edges in the base case):
* $\tilde{p}$ is the *unnormalized probability* of this set of paths (obtained by the `prob()` method)
* $\tilde{r} = \tilde{p} \bar{R} $ is the *unnormalized expected value* (obtained by the `value()` method)
* $\tilde{r}/\tilde{p} = \bar{R}$ is the *expected value* (obtained by the `expectation()` method)
By taking the concatenation ($\otimes$) of paths with edges and the union ($\otimes$) of complete paths, we can build up the expected value of an entire FSA.  The key to doing this efficiently is that the distributive property holds, so that we can use dynamic programming to get the total weight $\beta(\text{initial state})$ of all paths in the FSA.

In [ ]:
from expectation_semiring import ExpectationSemiringWeight  

In [ ]:
def fsa_edge(char, prob, reward):
    fsa = FST(ExpectationSemiringWeight, acceptor=True)
    start = fsa.add_state()
    end = fsa.add_state()
    fsa.initial_state = start
    fsa.set_final_weight(end)
    fsa.add_arc(start, end, 
                input_label=char, output_label=char, 
                weight=ExpectationSemiringWeight(prob, prob*reward))
    return fsa

def show(fsa):
    total = fsa.sum_paths()
    print(f'total weight {total} ==> expected value {total.expectation()}')
    display(fsa)

Let's check that $\oplus$ works as expected:

In [ ]:
a = fsa_edge('a',2,5)   # unnormalized probability of 2, reward of 5
b = fsa_edge('b',3,10)  # unnormalized probability of 3, reward of 10
show(a)
show(b)
show(a.union(b))        # 2/5 probability of getting reward 5, and 3/5 probability of getting reward 10

And let's check that $\otimes$ works as expected.  To understand the $\otimes$ computation, observe that in the second machine below, we have $\tilde{p} = 2 \cdot 3 \cdot 2$ (multiplied along the arcs), while $\tilde{r}$ is obtained in effect as $(2 \cdot 5) \cdot 3 \cdot 2$ from the first arc, plus $2 \cdot (3 \cdot 10) \cdot 2$ from the second arc, plus $2 \cdot 3 \cdot (2 \cdot 5)$ from the third arc.  This rearranges into $\tilde{r} = (2 \cdot 3 \cdot 2) \cdot (5+10+5) = \tilde{p} \cdot (5+10+5)$, so $\tilde{r}/\tilde{p} = 5+10+5 = 20$ as desired.

In [ ]:
ab  = a.concat(b);           show(ab)   # single path accepting 'ab'  with total reward of 5+10
aba = a.concat(b).concat(a); show(aba)  # single path accepting 'aba' with total reward of 5+10+5

<a name="ababa"></a>Finally, let's try $\oplus$ and $\otimes$ together.

In [ ]:
show(ab.union(aba))   # 6/18 chance of getting reward of 5+10, and 12/18 chance of getting reward of 5+10+5

We should get the same answer if we apply semantics-preserving transformations to the FSA.  Such transformations may move the weights around or change the topology, but still associate the same weight with every path.  Let's try pushing first, so that the total expected cost of `'ab'` and of `'aba'` are pushed back to their initial arcs:  

In [ ]:
show(ab.union(aba).push())

In the same spirit, we can find the minimal equivalent DFA, which in this case merges the common `'ab'` prefix.  Again we should get the same answer.  However, notice that this time we obtain the entire expected reward of `18.33` up front.  At state 2, we have 1/3 chance of stopping with `'ab'` and giving back `3.33` of the reward, and 2/3 chance of continuing on to `'aba'` and getting an additional `1.67` reward.  These effects cancel out.  

In [ ]:
# Minimization takes 3 successive OpenFST method calls, since minimization 
# requires a deterministic FSA, and determinization requires an epsilon-free FSA.  
# Feel free to play around with these steps separately.
show(ab.union(aba).remove_epsilon().determinize().minimize())

It is possible to compute expectations under $p$ of any quantity, not just the expected reward (gain):
* By defining $R(\boldsymbol{y})$ values to be a loss rather than a reward, you can compute the expected loss (risk).  This is just a change of sign.
* By defining $R(\boldsymbol{y})$ to be a vector $(\text{true_pos},\text{true},\text{pos})$ you can compute the expectation of that vector, $({\mathbb E}[\text{true_pos}],{\mathbb E}[\text{true}],{\mathbb E}[\text{pos}])$.  This is just computing three expectations at once.  Similarly, you could compute the expected rewards of several different plans.
* By defining $R(\boldsymbol{y})$ to be $\boldsymbol{y}$'s feature vector or more generally the gradient of its score, $\nabla G(\boldsymbol{y})$, you can compute the expectation of that feature vector or gradient.  In this case, the path weight is $\langle \tilde{p}(\boldsymbol{y}), \tilde{p}(\boldsymbol{y}) \nabla G(\boldsymbol{y}) \rangle$.  

Our implementation of `ExpectationSemiringWeight` therefore allows the second element to be a real number, a `numpy` array, a `ParamDict`, or pretty much any object that implements `+` and `*` operations.

In [ ]:
c = fsa_edge('c',2,np.array([1,2,3]))
d = fsa_edge('d',5,np.array([4,5,6]))
show(c.concat(d))

In [ ]:
c = fsa_edge('c',2,ParamDict(true=1,pos=2,true_pos=3))
d = fsa_edge('d',5,ParamDict(true=4,pos=5,true_pos=6))
show(c.concat(d))

#### Avoiding underflow/overflow

Multiplying probabilities along a long path risks underflow, or overflow if they are unnormalized probabilities.  

You already know the trick of switching from `RealSemiringWeight` to `LogSemiringWeight` (where $(\oplus,\otimes) = (\text{logsumexp}, +)$).  But the logsumexp operation is a bit slow.  And in the case of expectation semirings, the $\tilde{r}$ values may be as large or small as the $\tilde{p}$ values, but can't be straightforwardly represented in the log domain because they may be negative.  

Our `ExpectationSemiringWeight` class uses an alternative trick for avoiding underflow/overflow: it privately stores a scalar multiplier along with the $\langle \tilde{p},\tilde{r} \rangle$ pair.  This isn't unrelated to the log trick, since we actually store the *log* of the multiplier.  However, we ensure that this log is an integer, and we also store a multiplicand to account for the non-integer part and the sign.

In effect, this builds on the standard floating-point representation, which already has the form (mantissa, exponent), where "exponent" is the integer log of a possibly large multiplier, and "mantissa" is a multiplicand that is close to 1 or -1.  We continue to use floating-point representations for $\tilde{p}$ and $\tilde{r}$; it's just that our (shared) log-multiplier effectively extends the range of the exponent beyond what's possible in a standard 64-bit float.  To understand this engineering trick, check out the comments at the top of [expectation_semiring.py](expectation_semiring.py).

In [ ]:
w = ExpectationSemiringWeight(10,6)**50000   # enormous!
w                                            # printed representation makes use of scientific notation

In [ ]:
(w.prob(),          # far too large for a float
 w.value(),         # far too large for a float
 w.expectation(),   # but the ratio is manageable
 w.prob(log=True))  # and we are allowed to get the log(prob) to avoid overflow

Why is the expectation $\approx 30000$?

**Answer**: <span style='color:red'>FILL IN</span>

Why is the log(prob) $\approx 115129$?  (Note that the log is base $e$.)

**Answer**: <span style='color:red'>FILL IN</span>

#### Deeper discussion: The expectation semiring as a "gradient semiring"

When discussing expected gradient vectors above, we characterized the weight of path $\boldsymbol{y}$ as $\langle \tilde{p}(\boldsymbol{y}), \tilde{p}(\boldsymbol{y}) \nabla G(\boldsymbol{y}) \rangle$.  Observe that this can be interpreted as $\langle \tilde{p}(\boldsymbol{y}), \nabla \tilde{p}(\boldsymbol{y}) \rangle$ (since $\tilde{p} = \exp G$ by definition).  

This gives another interpretation of the expectation semiring as a simple device for tracking gradients.  A weight's first element is merely a real number with the usual operations from the real semiring, and its second element is always the gradient of the first element (with respect to some parameters).  It is easy to check that the semiring operations do preserve this property under addition, multiplication, and division of the weights.  The total weight of all paths must therefore be $\langle Z, \nabla Z \rangle$, and dividing the second element by the first element indeed gives the expected score gradient, since $\frac{\nabla Z}{Z} = \nabla \log Z = \mathbb{E}_{\boldsymbol{y}\sim p}[\nabla G(\boldsymbol{y})]$.  

But why should tracking gradients also be useful in computing expected *rewards*?  Well, with a little trickery, you can regard expected rewards as an example of expected feature values, and thereby get them from $\nabla \log Z$.  Redefine $\tilde{p}(\boldsymbol{y}) = \exp(G(\boldsymbol{y}) + R(\boldsymbol{y})\cdot \theta_{\text{reward}})$ where $R(\boldsymbol{y})$ is now used as a log-linear feature and $\theta_{\text{reward}}$ is its weight.  By setting $\theta_{\text{reward}}=0$, you can prevent the reward model from affecting the probability model at all.  Yet the partial derivative $\partial \log Z / \partial \theta_{\text{reward}}$ is still the expected feature value — that is, the expected reward you want!  This is another way to see why our expected reward computation worked.

But wait — if this is all about gradients, don't we have another way to compute gradients, namely automatic differentiation?  We do indeed.  In particular, you could just compute $Z$ in the real semiring (or the log semiring), but wrap each parameter as a PyTorch `Variable` (which mfst can handle by wrapping it further as `PythonValueSemiringWeight`).  Then `sum_paths` returns $Z$ as a `Variable`, and you could get $\nabla \log Z$ by backprop: `torch.log(Z).backward()`.  

Our method below for computing training gradients by forward-backward in the expectation semiring is actually isomorphic to this method.  So why did we bother implementing it, when PyTorch backprop is a highly optimized version of the same computation?  Well, the expectation semiring is interesting to think about, and perhaps more intuitive in the case of expected rewards.  It also doesn't require building a computation graph in memory; that graph is implicit in the FSA itself.  But the main reason is that our Bayes rule decoder will rely on manipulating an FSA in the expectation semiring.

## Part 4 — Using FSTs to decode our CRF model

To go beyond brute-force methods, we'll extend some of our homework 1 classes to specify FSTs (finite-state transducers).  This will give us access to the generic dynamic programming algorithms in OpenFST.

### FST task setting

Here's the extended interface that we'll have to implement:
<a name="TaskSetting"></a>

In [ ]:
class FstTaskSetting(TaskSetting):
    """
    Extend homework 1's TaskSetting class with methods that return FSTs (rather than iterators)
    to specify the relevant sets.  This permits efficient dynamic programming algorithms to
    replace the brute-force algorithms.
    """
            
    def fsa_yy(self, *, xx, oo=None):
        """
        As an alternative to `iterate_yy`, return an unweighted FSA in the BooleanSemiring
        that defines the valid output sequences `yy` for input `xx`.  (Recall that in the  
        OpenFST toolkit, an FSA is implemented as an "acceptor" FST where on each arc, the 
        input and output symbols are equal.)
        """
        raise NotImplementedError()
         
    def fsa_aa(self, *, xx):
        """
        As an alternative to `iterate_aa`, return an unweighted FSA in the BooleanSemiring
        that defines the valid plan sequences `aa` for input `xx`.
        The default (as for `iterate_aa`) is to return just the valid output sequences.
        """
        return self.fsa_yy(xx=xx)
    
    # We can get an iterator for free from the FSA, so to specify a particular task,
    # it's enough to specify only the FSA and not the iterator.
    def iterate_yy(self, *, xx, oo=None):
        for path in self.fsa_yy(xx=xx, oo=oo).iterate_paths():
            yield tuple(path.input_path)

    def iterate_aa(self, *, xx):
        for path in self.fsa_aa(xx=xx).iterate_paths():
            yield tuple(path.input_path)
    
    def fst_reward(self, *, xx):
        """
        As an alternative to `reward`, return an FST in the MaxPlusSemiring 
        that defines the reward function.   For every output string `yy` 
        and every plan `aa`, the FST should transduce `yy` to `aa` with weight R(a | x, y).
        (Note that `yy` is the *input* to this FST.)
        
        This FST may accept illegal `yy` values since the probability model
        will give them probability 0 anyway.  It may also produce illegal `aa`
        values; it is up to the caller to compose it with `fst_aa`.  This allows
        `fst_aa` to be overridden, and may also simplify the `fst_reward` machine.
        """
        raise NotImplementedError()
        
    # In principle, we should here define generic implementations of the `TaskSetting` methods
    # `iterate_yy`, `iterate_aa`, and `reward` that use the finite-state machines above.
    # (These could still be overridden by simpler and faster direct implementations.)
    # However, we can get away without these methods for purposes of this homework.

Now, your job is to extend `IobTask0` into an FST-enabled version.  
This might be the trickiest part of the assignment.
Here are some hints.

For `fsa_yy`, you could build the FSA by specifying all its states and arcs explicitly.
Or more elegantly, you could build the FSA from simpler machines using finite-state
operations, as well as existing utility functions like `create_from_string` and 
`create_from_observable`.

You have a similar choice with `fst_reward`.  However, it turns out to be remarkably 
tricky to define this machine in a declarative way, especially if you want to make
it small (since the finite-state determinization and minimization constructions don't
always apply).  Thus, we suggest constructing this machine by hand, to mimic a procedure 
where you read the `(yy,aa)` pair from left to right, adding a reward of `1` whenever
you encounter a true positive and a reward of `-self.false_pos_penalty` whenever you
encounter a false positive.  Our old `reward` function did something like this.

You should be able to do this with just two states.  You enter the "match" state upon
reading `B:B`.  Being in the "match" state means that `yy` and `aa` *started* proposing a chunk in 
the same place; they continue to match as long as you read `I:I`.  Depending on whether these two chunks also *end*
in the same place, you may discover that they form a true positive, or a false positive 
and a false negative.  It is also possible to spot false positives and false negatives 
while in the "nomatch" state.  Make sure that you can read all possible symbol pairs 
(and also that you can stop) when in either state, and that you score these events correctly!
The code can be made pretty concise.

In [ ]:
class IobTask(IobTask0, FstTaskSetting):    
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._fsa_legal_yy = None  # cached machine that doesn't vary by call
        self._fst_reward = None    # cached machine that doesn't vary by call (but it does vary by instance of the task, because of different false_pos_penalty values)
    
    def fsa_yy(self, *, xx, oo=None):
        """
        An FSA in the BooleanSemiring that accepts just the legal IOB sequences for `xx` — 
        the ones that would be returned by `IobTask0.iterate_yy`.  Make sure to consider `oo`.
        """
        # Hint: use `self.Y_alphabet`.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
                
    def fst_reward(self, *, xx):
        """
        An FST that can read *any* legal `yy`:`aa` pair, 
        and assign it the same weight that `IobTask0.reward` would have computed, 
        which depends on the number of true positives, the number of false positives,
        and `self.false_pos_penalty`.
        """
        # Since the reward for the IOB task does not happen to depend on xx,  
        # we can reuse the reward FST from last time if we've already built it.
        if self._fst_reward:
            return self._fst_reward
        
        # Compute `self._fst_reward` before returning it.
        # Make sure to use `MaxPlusSemiringWeight`.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


In [ ]:
task = IobTask()
fivewords = ('een','twee','drie','vier','vijf')   # a five-word Dutch "sentence" (= one, two, three, four, five)

In [ ]:
task.fsa_yy(xx=fivewords)

In [ ]:
task.fsa_yy(xx=fivewords, oo='?O???')

In [ ]:
# confirm that fsa_yy describes the same set of strings as the original iterate_yy
list(task.iterate_yy(xx=fivewords))
assert (   set(task .iterate_yy(xx=fivewords))    # enumerates paths in fsa_yy
        == set(task0.iterate_yy(xx=fivewords))  ) # hand-crafted iterator
assert (   set(task .iterate_yy(xx=fivewords,oo='?O???')) 
        == set(task0.iterate_yy(xx=fivewords,oo='?O???')) )

In [ ]:
fst_R = task.fst_reward(xx=fivewords) 
fst_R

In [ ]:
# confirm that fst_R gives the same answer on ALL (yy,aa) pairs as the original reward() -- an extensive test!
for yy in task.iterate_yy(xx=fivewords):
    for aa in task.iterate_aa(xx=fivewords):
        R = FST(''.join(yy)).compose(fst_R, FST(''.join(aa))).sum_paths()   # finite-state computation of R(aa | yy)
        R0 = task.reward(aa=aa,xx=fivewords,yy=yy)               # old computation of R(aa | yy)
        assert math.isclose(R,R0), f'fst reward={R}, true reward={R0}, F1_triple={task.reward_F1_triple(aa=aa,xx=fivewords,yy=yy)}, yy={yy}, aa={aa}'

### FST probability model

Given $\boldsymbol{x}$, our CRF defines a trellis of possible $\boldsymbol{y}$ values.  We will express this as a 
weighted FSA that maps each $\boldsymbol{y}$ sequence to its unnormalized probability $\tilde{p}(\boldsymbol{y} \mid \boldsymbol{x}) = \exp G(\boldsymbol{x},\boldsymbol{y})$.  

So just as we earlier extended `TaskSetting` into `FstTaskSetting` with extra methods `fsa_yy` and `fst_reward`, we will now extend `BoltzmannModel` into `FstBoltzmannModel` with an extra method `fsa_uprob`.  

We'll also override some of the `BoltzmannModel`'s existing methods with more efficient versions that make use of `fsa_uprob` with dynamic programming, instead of repeatedly calling `score` and `score_with_gradient`.  Fill in the missing methods below, based on work you've already done earlier in the notebook.  You should probably look back at `BoltzmannModel` from homework 1.

Given $\boldsymbol{x}$, our CRF defines a trellis of possible $\boldsymbol{y}$ values.  We will express this as a 
weighted FSA that maps each $\boldsymbol{y}$ sequence to its unnormalized probability $\tilde{p}(\boldsymbol{y} \mid \boldsymbol{x}) = \exp G(\boldsymbol{x},\boldsymbol{y})$.

So just as we earlier extended `TaskSetting` into `FstTaskSetting` with extra methods `fsa_yy` and `fst_reward`, we will now extend `BoltzmannModel` into `FstBoltzmannModel` with an extra method `fsa_uprob`.  We'll also override some of the `BoltzmannModel`'s existing methods with more efficient versions that make use of `fsa_uprob` with dynamic programming, instead of repeatedly calling the old `score` and `score_with_gradient` methods.

In [ ]:
class FstBoltzmannModel(BoltzmannModel):
    """ 
    Extend homework 1's BoltzmannModel class with methods that can return a weighted FST 
    to score the yy values.  This permits efficient dynamic programming algorithms to
    compute the normalizer and its gradient without brute-force enumeration of the yy values.
    """
    
    def fsa_uprob(self, *, xx, temperature=1, include_gradient=False):
        """
        Given input `xx`, return a weighted FSA such that for any legal output `yy`,
        the FSA accepts the sequence `yy` with `RealSemiringWeight` of
        p̃(yy | xx) = exp G(xx,yy). (Note that this may overflow; we should
        really have a flag saying to work in the log domain.)
        
        The FSA should not accept any `yy` outputs that are illegal according 
        to `self.task.fsa_yy(xx)`.
        
        If the include_gradient flag is set, then the weight should instead be
        (p̃, ∇p̃), which is an ExpectationSemiringWeight.  This version should not
        overflow.  Note that the .expectation() method of this weight returns 
        ∇p̃/p̃ = ∇(log p̃) = ∇G (which is the feature vector f(xx,yy) in the 
        log-linear case G=θ⋅f).
        
        (What if `temperature != 1`?  It's not hard to see that the feature
        vector should then be divided by the temperature throughout, including
        in the gradient.)
        
        The FSA over `yy` values may be constructed on demand given `xx`.  However, 
        for some models, one might also be able to obtain the FSA by finite-state 
        composition, as `FST(xx).compose(crf).project(type='output')`, where `crf` 
        is a fixed static FST representing the entire CRF model.
        """
        raise NotImplementedError()

    def normalizer(self, *, xx, oo=None, temperature=1):
        fsa = self.fsa_uprob(xx=xx, temperature=temperature)
        if oo is not None:
            fsa = fsa.compose(self.task.fsa_yy(xx=xx, oo=oo))
        return fsa.sum_paths().value
        
    def logprob_gradient(self, *, xx, oo=None, yy=None):
        # Compute the gradient of the clamped (numerator) term 
        # minus the gradient of the free (denominator) term
        # to get p(yy | xx) or p(oo | xx).
        # Hint: This is what the expectation semiring was *for*!
        # Hint: Use `task.fsa_yy(xx=xx,oo=oo)` to restrict numerator to `oo`.
        assert (oo is None) != (yy is None)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END

    def sampler(self, *, xx, oo=None, temperature=1):
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


**Remark**: Recall that `score(xx,yy)` and `score_with_gradient(xx,yy)` are fast.  They do not have to build a whole `fsa_uprob(xx)` just to get the score of the `yy` path.  

For this reason, we did not override them with FST versions, nor did we override `uprob`.  Recall that `uprob(yy)` just calls the fast exp(`score`).  (And `uprob(oo)` calls `normalizer`, which has already been overridden with an FST version.)  

But as a matter of class design, does this mean that an implementor of `FstBoltzmannModel` is *still required to implement* `score` and `score_with_gradient`?  This seems annoying when they have already put the same information is in `fsa_uprob`.
* Fortunately, there exist straightforward general implementations of `uprob`, `score`, and `score_with_gradient` in terms of `fsa_uprob`.  So these could be provided by a mix-in class.  
* Or perhaps these old methods could simply be omitted since they won't be called.  The trainer and decision agent should normally bypass them in favor of methods like `logprob_gradient` that use the FST.  The main reason we have the old methods at all is to compare them to our new methods.

Now extend `BigramCrfModel0` to support an FSA by defining `fsa_uprob`.  
* The topology will look a lot like `build_HC_fsa`  from above, except that you should restrict it to tag sequences that are legal for the task at hand.  (This is specified by `task.fsa_yy` — as we mentioned before, `BigramCrfModel` is a *general* tagger that could be used for tasks other than IOB tagging!)
* The arc weights will be computed as in `BigramCrfModel0.score_with_gradient`.  That is, instead of using fixed parameters `same_weight` and `change_weight`, each arc and each final state will have a positive real weight $\psi$, which is determined by about 3 features whose weights live in `params`.  If you are including the gradient, the weight should be $\langle \psi, \nabla \psi \rangle$, where $\nabla \psi$ is $\psi$ times that (very sparse) feature vector.  It will be convenient to build that feature vector in any case.

In [ ]:
class BigramCrfModel(BigramCrfModel0, FstBoltzmannModel):  
    
    def fsa_uprob(self, *, xx, temperature=1, include_gradient=False):
        
        def weight(features):
            """
            Convert a feature vector to a weight in the appropriate semiring, 
            using `self.params` and `temperature`.
            """
            ### STUDENTS START
            raise NotImplementedError()  # REPLACE ME
            ### STUDENTS END
        
        fsa = FST(ExpectationSemiringWeight if include_gradient else RealSemiringWeight, 
                  acceptor=True)
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END
        return fsa.intersect(task.fsa_yy(xx=xx))   # restrict to taggings that are legal for `task`

Now we can create `model` that is just like `model0` but has the extra FST methods.  Since we already trained `model0`, we'll just steal a copy of its parameters (for now) rather than retraining.

In [ ]:
class TrainableBigramCrfModel(L2LogLikelihood, BigramCrfModel): 
    pass   # inherit everything from the two parents

model = TrainableBigramCrfModel(task, 
                                vocab=vocab_small, 
                                regularization_coeff=15,
                                num_examples=len(data_small))
model.params = model0.params

To check our FSA and finite-state methods, let's first make sure that `fsa_uprob` and `normalizer` are getting the same probabilities and feature vectors on short dev sentences that we got with the simple `uprob` and `normalizer` methods. 

In [ ]:
def compare_models(model,model0,dataset):   # model is an FstModel, but model0 uses the old homework 1 methods

    for xx,oo,yy in iterate_data(dataset):
        fsa     = model.fsa_uprob(xx=xx)
        fsagrad = model.fsa_uprob(xx=xx, include_gradient=True)
        # display(fsa)   # uncomment for debugging
    
        uprob    = fsa    .compose(FST(yy)).sum_paths().value          # weight of yy in the FSA
        gradient = fsagrad.compose(FST(yy)).sum_paths().expectation()  # feature vector for computing that weight

        uprob0      = model0.uprob(xx=xx, yy=yy)                  
        _,gradient0 = model0.score_with_gradient(xx=xx, yy=yy)    
        assert math.isclose(uprob, uprob0), f'p̃(yy | xx) = {uprob} (new), {uprob0} (old)\n\t{gradient}\n\t{gradient0},\n\t{gradient - gradient0}'
    
        if oo:
            uprob  = model.normalizer(xx=xx, oo=oo)
            uprob0 = model0.normalizer(xx=xx, oo=oo)
            assert math.isclose(uprob, uprob0), f'p̃(oo | xx) = {uprob} (new), {uprob0} (old)'
    
        Z = model.normalizer(xx=xx)
        Z0 = model0.normalizer(xx=xx)   # the brute-force method that we overrode
        assert math.isclose(Z,Z0), f'Z(xx) = {Z} (new), {Z0} (old)'
        
compare_models(model, model0, 'dev-small')   # asserts that models match

Once it works, let's try training with the new model!  
(Is it faster than before?  Probably not, because these sentences are still too short for the FSA to beat brute-force.)

In [ ]:
sgd_trainer = SGDTrainer(epochs=10)
%time sgd_trainer.train(model, data_small)

Hopefully you learned exactly the same params as before.  You could check by comparing `model.params` with `model0.params`, but let's just compare the predictions of the parameters, exactly as we did before retraining `model.params` with the new algorithms:

In [ ]:
compare_models(model, model0, 'dev-small')

Oh, and let's compare the results of sampling ...

In [ ]:
def samples(model,xx,oo=None,n=1000):
    sampler = model.sampler(xx=xx,oo=oo)   # exact sampler
    counts = ParamDict()             # not actually using this for params
    for yy in islice(sampler,n):
        counts[''.join(yy)] += 1
    return counts

xx = ('brussel', '/', 'den', 'haag')   # a dev sentence whose correct tagging is BOBI
n = 100000
%time s  = samples(model,  xx, n=n)
%time s0 = samples(model0, xx, n=n)
(s-s0)/n     # difference in estimated probabilities: hopefully small (could do hypothesis testing!)

Just to prove that the finite-state method really was worth implementing for speed, let's try a longer sentence.

In [ ]:
xx = xx * 3    # 12-word sentence
n = 10
%time s  = samples(model,  xx, n=n)
%time s0 = samples(model0, xx, n=n)

Linear-time really does beat exponential-time, especially on longer sentences.  Even one long sentence in the corpus would greatly slow down the brute-force algorithms

### Forward-backward for training

You've been using the forward algorithm (`sum_paths`) to compute the total weight of `fsa_uprob`.  When the FSA is weighted by the expectation semiring, this is rather slow.  The reason is that every $\oplus$ operation involves a vector sum, and every $\otimes$ operation involves a linear combination of vectors.   

Moreover, these are unfortunately *dense* vector operations.  Basically, you are sweeping from left to right through the FSA, adding up the features on the arcs as you encounter them.  For a state $s$ at time step $i$, the forward weight $\alpha(s)$ includes a vector that is dense with all features encountered on all the length-$i$ paths to that state.  

Remember, however, that the forward-backward algorithm is another way to compute expectations and gradients.  It turns out that `sum_paths` for machines in the expectation semiring can be computed faster by the forward-backward algorithm.  Here, instead of pushing feature vectors from the initial state to the final state, the feature vectors stay where they are, and we push scalar probabilities toward them from both sides to determine how heavily to weight them in the final sum.

Both algorithms have the same asymptotic runtime, in the sense that on a machine with $m$ arcs, they both perform $O(m)$ multiply-adds, in the sense of multiplying a vector by a scalar and adding it to a total.  However, these vectors tend to be dense in the forward algorithm — in the forward-backward algorithm they are just the feature vectors attached to individual arcs, which are usually quite sparse.

#### The algorithm

Specifically, suppose each arc $e$ in the machine is labeled with an expectation semiring weight $\langle \psi_e, \boldsymbol{v}_e \rangle$, and its source and destination states are $s_e$ and $t_e$.  Each final state $f$ has a final weight of $\langle \psi_f, \boldsymbol{v}_f \rangle$.  

Temporarily ignore the $\boldsymbol{v}_e$ part of each weight so that our machine appears to be in the real semiring.  Run forward-backward over that simpler machine to obtain the forward probabilities $\alpha(s)$, the backward probabilities $\beta(t)$, and the sum of paths $Z = \beta(\text{initial state})$.  

Now the total weight of all paths in the original machine is $$\langle Z, \sum_e \alpha(s_e) \cdot \boldsymbol{v}_e \cdot \beta(t_e) \rangle + \sum_f \alpha(f) \boldsymbol{v}_f$$
where the second term is simply a weighted sum of the (sparse) feature vectors on the edges.  This is really just the standard way of collecting expected counts during the forward-backward algorithm.  (You might have expected to see a factor of $\psi_e$ in the sum, but $\boldsymbol{v}_e$ has already been weighted by $\psi_e$.)

#### Deeper discussion

It turns out that the forward algorithm is a special case of *forward-mode automatic differentiation*, which computes the objective function $F(\boldsymbol{\theta})$ and its gradient in parallel using a single forward pass through the computation graph.  As you know from our previous discussion of the expectation semiring, it replaces each node $x$ in the computation graph with a pair $\langle x, \nabla_{\boldsymbol{\theta}} x \rangle$, so that at the end, it has found $\langle F, \nabla_{\boldsymbol{\theta}} x \rangle$.  The trouble is that each $\nabla_{\boldsymbol{\theta}} x$ is an entire vector in $\mathbb{R}^K$ (all values $\partial x / \partial \theta_k$),where $K$ is the number of parameters.

By contrast, the forward-backward algorithm is a special case of *reverse-mode automatic differentiation*, which uses both a forward pass to compute $F$ and a backward pass to compute its gradient.  The backward pass augments each node $x$ with an adjoint, which is just a scalar ($\partial F / \partial x$) provided that $F(\boldsymbol{\theta})$ is a scalar.  

In short, for a function $F: \mathbb{R}^K \rightarrow \mathbb{R}$, reverse-mode is much better.  (Forward-mode does win for a function $\mathbb{R}^K \rightarrow \mathbb{R}^{K'}$ where $K' \geq K$, which is actually the case when we are computing expected reward.)

#### Implementing forward-backward

Let's try it out using our 12-word sentence from above:

In [ ]:
fsa = model.fsa_uprob(xx=xx, include_gradient=True)
ZZ = fsa.sum_paths()
ZZ

Here's how we can move our FSA into the real semiring so that we can run ordinary fast forward-backward on it.  The `lift()` method makes a copy of the FSA but maps the weights into a new semiring using a user-provided function.

In [ ]:
fsa_real = fsa.lift(RealSemiringWeight, lambda w: w.prob())
fsa_real

In [ ]:
# OpenFST uses the term "shortest distance" to mean the semiring generalization of shortest
# distance, namely the total probability of all prefix paths to a state or suffix paths from a state.
alpha = fsa_real.shortest_distance() 
beta = fsa_real.shortest_distance(reverse=True)
Z = beta[fsa_real.initial_state]
for s in fsa_real.states:
    print(s, alpha[s]*beta[s] / Z, alpha[s], beta[s])  # marginal probabilities of being in different states

To avoid numerical issues during forward-backward, it's better to lift the weight $\langle \psi,\boldsymbol{v}\rangle$ not into the real semiring as $\psi$, but into the log semiring as $\log \psi$.  
Or we can lift it into the expectation semiring as $\langle \psi,0 \rangle$, since such a weight behaves like $\psi \in \mathbb{R}$, and won't overflow/underflow thanks to our cool implementation.  
We'll do the latter, using a special `dropvalue` method that maps $\langle \psi,\boldsymbol{v}\rangle \mapsto \langle \psi,0\rangle$.

In [ ]:
def sum_paths_fb(fst):
    "Use forward-backward to find the pathsum of an FST in the expectation semiring." 
    assert fst.semiring is ExpectationSemiringWeight
    fst_real = fst.lift(ExpectationSemiringWeight, converter=lambda w: w.dropvalue())
    alpha = fst_real.shortest_distance() 
    beta  = fst_real.shortest_distance(reverse=True)
    Z = beta[fst_real.initial_state]

    total = ExpectationSemiringWeight(0)
    for s in fst.states:
        # if s is final, then get_arcs will yield an arc to state -1 with the final-state weight
        for e in fst.get_arcs(s):
            multiplier = alpha[s] * (beta[e.nextstate] if e.nextstate >= 0 else ExpectationSemiringWeight.one)
            total += multiplier * e.weight     # avoid multiplying the big `e.weight` by alpha and beta separately 
    # The second element of total will now be correct, but we need to replace the first element with Z.
    # Here's a slightly hacky approach that remains safe (even if total and Z are encoded with different multipliers).
    return total + (Z.dropvalue() - total.dropvalue())

Let's confirm that it works.  You may also want to play around with the [machines from the expectation semiring section](#ababa) to get a better intuition for why it works.

In [ ]:
assert sum_paths_fb(fsa).approx_eq(ZZ)

**Implementation:** To activate this special handling, we'll simply patch `FST.sum_paths` to notice when the FST is in the expectation semiring, and use the faster algorithm then.

In [ ]:
FST.expectation_uses_fb = True       # control whether we use the fast method

def sum_paths_with_fb(self, *args, **kwargs):
    if FST.expectation_uses_fb and self.semiring is ExpectationSemiringWeight:
        return sum_paths_fb(self, *args, **kwargs)
    else:
        return self._sum_paths(*args, **kwargs)
    
if not hasattr(FST,'_sum_paths'):
    FST._sum_paths = FST.sum_paths   # save the original method
FST.sum_paths = sum_paths_with_fb    # replace the method dynamically

Let's try training again, this time using forward-backward.  Again, for these tiny sentences, the speedup won't be apparent.

In [ ]:
sgd_trainer = SGDTrainer(epochs=10)
%time sgd_trainer.train(model, data_small)

But how about the full dataset?  We'll have to replace our `model` with one that has a larger vocabulary and hence a larger feature set.  Adapting a bit of code from before:

In [ ]:
data_large = list(iterate_data('train'))
vocab_large = vocab_from_data(data_large)
(len(vocab_large), list(islice(vocab_large,10)))  # size of vocab and 10 words from it

Let's compare the speed of the forward strategy versus forward-backward.  The model has about 30000 features, most of which are not used on any given sentence.  So if you use a dense vector class such as `numpy.array`, you will be spending a lot of time rapidly adding multiples of zero together.  Instead a sparse vector class such as `ParamDict` should save some time.  And once you're using such a sparse vector class, the forward-backward strategy should help (especially on longer sentences).

In [ ]:
FST.expectation_uses_fb = False

model = TrainableBigramCrfModel(task, 
                                vocab=vocab_large, 
                                regularization_coeff=15,
                                num_examples=len(data_large))
sgd_trainer = SGDTrainer(epochs=1)
%time sgd_trainer.train(model, data_large[0:500])

In [ ]:
FST.expectation_uses_fb = True

model = TrainableBigramCrfModel(task, 
                                vocab=vocab_large, 
                                regularization_coeff=15,    # you could play with the regularizer
                                num_examples=len(data_large))
sgd_trainer = SGDTrainer(epochs=1)
%time sgd_trainer.train(model, data_large[0:500])

Ok, let's train a real model that we will use for the rest of the homework.  Go get something to eat because this could take an hour ... but you should only have to do it once.  (Once for now, anyway.  You might want to tune it some more at the end of the notebook, before running your final test.)

In [ ]:
sgd_trainer = SGDTrainer(epochs=5)           # constant stepsize; you could play with the stepsize and its decay rate
%time sgd_trainer.train(model, data_large)   # keep on training current model (the trainer will shuffle the dataset)

In [ ]:
len(model.params)

### FST Viterbi decision agent

Now that we've trained our model, a natural thing to do is to see how well a simple Viterbi decision agent can fare.  However, it will be rather slow on these full-length sentences!  Let's try just the *very first* dev sentence ...

In [ ]:
agent1 = ViterbiAgent(task, model)
%time agent1.test_F1(islice(iterate_data('dev'),1))

Hmm, you probably had to interrupt the notebook kernel on that one.  (It's a 24-word sentence with only 3 observed tags, so brute force isn't going to terminate any time soon.)  

But you could try evaluating the same agent on 'dev-small'.

In [ ]:
agent1 = ViterbiAgent(task, model)
agent1.test_F1(iterate_data('dev-small'))

Unfortunately, we do even worse here than our earlier `agent0` with `model0`.  That's because `'dev-small'` is not the kind of data that `model` was trained to work on — a case of "domain mismatch."  (In particular, notice that `model` has learned to predict relatively few chunks ... far too few for `'dev-small'`.  As a result, it gets very low recall.  It also gets lower precision than `model0`.)

We'd better figure out how to test on our actual data domain.  Let's extend our agent to use FSTs, just as we extended our task setting and our model!  You have enough tools to fill this in.  

Properly speaking, your `FstViterbiAgent` will find the *path* of highest model probability (given `xx`,`oo`) and report its string `yy`.  This is the traditional Viterbi decoding strategy.  Why might it not necessarily find the highest-probability *string*?  What could you do to fix that (and how expensive would it be)?  Will it find the highest-probability string when it's applied to `FstBigramCrfModel`?  Why or why not?

**Answer**: <span style='color:red'>FILL IN</span>

In [ ]:
class FstViterbiAgent(ViterbiAgent):

    def decision(self, *, xx, oo=None):
        # Hint: Start with `fsa_uprob` and `lift` it into the tropical semiring to avoid underflow
        # Hint: You're trying to choose the `yy` with highest posterior probability since you are 
        #        a Viterbi agent; info about which `yy` are compatible with `oo` is given by the 
        #        task setting, not the model.
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


In [ ]:
agent = FstViterbiAgent(task, model)
agent.test_F1(iterate_data('dev-small'))

In [ ]:
for xx,oo,yy in iterate_data('dev-small'):
    yy = agent.decision(xx=xx,oo=oo)     # FST
    yy1 = agent1.decision(xx=xx,oo=oo)   # brute-force
    assert yy == yy1, f'yy = {yy}, yy1 = {yy1}, oo = {oo}, xx = {xx}'

If you got the same answers as before on `'dev-small'`, now try it on `dev`.

In [ ]:
%time agent.test_F1(iterate_data('dev'))

Well, that's a lot faster, but the recall on the 1-best path is still way too low.  Since `pos` is only 15, this implies that the single best path rarely has any chunks.  On at least 185 of the 200 dev sentences, the best path was just `OOOO...`.  

### FST Bayes rule agent

A smarter agent would realize that the low recall is killing the F1 score.  To get F1 score up, it had better take a risk and predict some entities.  The safest entities to guess would be those with a high *marginal* probability, which means summing over paths.

This is what a Bayes rule will do.  As we saw in homework 1, it will try to maximize our expected reward, or at least our expected [proxy reward](#proxy).  

The brute-force Bayes agent from homework 1 looped over the exponentially large set of `(aa,yy)` pairs to compute the Bayes decision $$ \text{argmax}_a \mathbb{E}_{y\sim p(\cdot | x)}[R(a | x, y)] $$
But here's a sketch of a much faster algorithm for computing the optimal chunking under our particular proxy reward function, given our particular model:

* Observe that in a sentence of length $J$, we have $O(J^2)$ possible chunks.  In principle, we can find the marginal probability of each of chunk.  For example, what is the marginal probability of a length-5 chunk starting at word 12?  Such a chunk corresponds to a `BIIII` substring that starts at word 12 and is not immediately followed by another `I`.  It's not too hard to see how to find the total weight of all such paths, with the help of pre-computed $\alpha(s)$ and $\beta(t)$ quantities to sum over the parts before and after `BIIII`.  A first guess is that it takes $O(J)$ time to score each of the $O(J^2)$ chunks, but maybe you can see how to do it in $O(J^2)$ time altogether.
* If the marginal probability of a chunk is $p$, then the expected reward of including the chunk is $p \cdot 1 + (1-p) \cdot (-\lambda)$.  We would prefer to include the chunk if this is positive, i.e., if $p \geq \frac{\lambda}{1+\lambda} \in (0,1)$.  (This fits with the idea that low $\lambda$ induces us to predict more chunks.)
* However, overlapping chunks are not allowed.  Thus, we must select a subset of non-overlapping chunks with the maximum *total* expected reward.  This is itself a dynamic programming problem.  (We might prefer to omit the most probable chunk if that would let us take two or more slightly less probable chunks.)  Maybe you can see how to handle this DP problem as finding the highest-weighted path in a certain acyclic graph, i.e., a problem in the $(\max,+)$ semiring.  The graph has $O(J^2)$ edges — one per chunk — and thus this stage of the algorithm also takes $O(J^2)$.

Moreover, we don't need to implement this algorithm from scratch or even design it!  The algorithm will pretty much fall out of a finite-state recipe, cooked with FSTs that are already in our pantry.

#### Exploring an example

Let's illustrate with a small example.  It has 1 true chunk, `radio roxy`, but the Viterbi path has 0 chunks:

In [ ]:
xx,oo,yy = list(iterate_data('dev'))[25]
xx    

In [ ]:
model.fsa_uprob(xx=xx)

In [ ]:
yy_hat = agent.decision(xx=xx)
yy_hat, model.prob(xx=xx, yy=yy_hat)    # Viterbi decision and its probability

In [ ]:
yy, model.prob(xx=xx, yy=yy)            # true yy and its probability 

The basic idea is to construct a single FSA in the (max,+) semiring that scores the different $\boldsymbol{a}$ sequences according to their expected rewards.  

We start with an FST that assigns to each $\boldsymbol{y}:\boldsymbol{a}$ pair the weight $\langle \tilde{p}, \tilde{p}r \rangle$ in the expectation semiring, where 
* $\tilde{p} = \tilde{p}(\boldsymbol{y} \mid \boldsymbol{x})$ or $\tilde{p} = \tilde{p}(\boldsymbol{y} \mid \boldsymbol{x},\boldsymbol{o})$
* $r = R(\boldsymbol{a} \mid \boldsymbol{x},\boldsymbol{y})$

To construct this FST, we just have to lift `fsa_uprob` from the $(+,\times)$ semiring to the expectation semiring (via $\tilde{p} \mapsto \langle \tilde{p}, 0 \rangle$) and lift `fst_reward` from the $(\max,+)$ semiring to the expectation semiring (via $r \mapsto \langle 1,r \rangle$)

Then composing these lifted machines in the expectation semiring will combine an `fsa_uprob` path that accepts $\boldsymbol{y}$ with probability $\tilde{p}$ with an `fst_reward` path that transduces $\boldsymbol{y}$ to $\boldsymbol{a}$ with reward $r$.   For the weight of this combined path, it multiplies the lifted versions of these weights to yield $\langle \tilde{p}, 0 \rangle \otimes \langle 1, r \rangle = \langle \tilde{p}, \tilde{p}r \rangle$ as desired.  

This happens for each $\boldsymbol{y}:\boldsymbol{a}$ pair, but structure is shared among pairs in the usual way, so the resulting FST is compact.  Note that `fsa_uprob` has $\leq 3J$ states and `fst_reward` has $2$ states, so the composed machine will have $\leq 6J$ states.

In [ ]:
fsa_uprob_lifted =  model.fsa_uprob(xx=xx)       \
                    .lift(ExpectationSemiringWeight, lambda p: ExpectationSemiringWeight(p.value,0))
fst_reward_lifted = task.fst_reward(xx=xx)       \
                    .lift(ExpectationSemiringWeight, lambda r: ExpectationSemiringWeight(1,r.value))
fst_both = fsa_uprob_lifted.compose(fst_reward_lifted)

print(fsa_uprob_lifted.num_states, fst_reward_lifted.num_states, fst_both.num_states)
fst_both

#### Evaluating plans

We would now like to marginalize over the $\boldsymbol{y}$ values, so that the machine will accept each $\boldsymbol{a}$ with its expected reward:

In [ ]:
fsa_reward = fst_both.project('output')   # drop the input (yy) labels, 
                                          # leaving the output (aa) labels and weights unchanged

Let's look at the expected rewards of a couple of selected plans `aa` -- the optimal plan and the Viterbi plan:

In [ ]:
aa_star = yy                # the optimal plan is the true tagging (which we can't know without peeking)
aa_top  = yy_hat            # the Viterbi plan is the most probable tagging (computed before)

def expected_reward(aa, trace=False):
    total = fsa_reward.compose(FST(aa)).sum_paths() 
    expectation = total.expectation()
    if trace:
        print(f'paths with aa={aa} ==> total weight {total} ==> expected reward {expectation}')
    return expectation
    
expected_reward(aa_star, trace=True)
expected_reward(aa_top, trace=True)
print(f'Z = {fsa_uprob_lifted.sum_paths()}')

Regardless of `aa`, the total unnormalized probability of the many paths mapping the various `yy` to `aa` is the same.  It is just $Z$, the total unnormalized probability of `yy` values, which is *unchanged by the choice of `aa`*.  (That is because our decision agent's action sequence `aa` does not affect which chunks `yy` the environment has actually placed there -- in contrast to a reinforcement learning agent, whose actions can affect the environment.)

However, each `aa` has different expected reward, obtained by summing over those various `yy` paths.  As shown above, this is computed in the expectation semiring.

Why does `aa_star` have expected reward in the range $(-1,0)$?

**Answer**: <span style='color:red'>FILL IN</span>

Why does `aa_top` have expected reward of exactly 0?

**Answer**: <span style='color:red'>FILL IN</span>

In fact, there are no paths with strictly positive expected reward, so our Bayes rule should just pick `'OOOOO'`, like Viterbi:

In [ ]:
# Look for good plans `aa` according to `fsa_reward`
for aa in task.iterate_aa(xx=xx):
    if expected_reward(aa) >= 0:
        expected_reward(aa, trace=True)

But what if we changed the false positive penalty $\lambda$ to 0?  Then *all* paths except for `'OOOOO'` would surely have positive expected reward (since they posit one or more chunks which have *some* chance of being correct, and *no* penalty for being wrong).  So there must be some intermediate values of $\lambda$ that also have paths of positive expected reward.  With such a $\lambda$, our Bayes rule would pick something other than `'OOOOO'`.  We'll be trying this below.

#### Determinization

At any rate, we have one more step.  Above we used a brute-force loop to look for good plans `aa`.  But we would like to do that by dynamic programming.

We want the `aa` with *maximum* expected reward according to `fsa_reward`.  The problem is that this reward is partitioned over paths.  (Remember that `fsa_reward` was obtained by stripping `yy` labels from `fst_both`.  Some paths accepting `aa` in `fst_reward` have positive reward and some have negative reward, depending on the original `yy` labels.)

To consolidate all weights for each `aa` path, we can *determinize* the `fsa_reward` machine, which takes weights in the expectation semring.  Like this:

In [ ]:
fsa_reward_det = fsa_reward.determinize()

The weight of each `aa` is exactly the same as we reported before.  In particular, the total weight of each path has the form $\langle Z, Z\bar{r} \rangle$ where $\bar{r}$ is the expected reward of `aa`.  Now we just have to find the plan of maximum expected reward.  We can do this by lowering `fsa_reward_det` back into the $(\max,+)$ semiring and finding the shortest path:

In [ ]:
fsa_reward_maxplus = fsa_reward_det \
                     .lift(MaxPlusSemiringWeight, lambda w: w.expectation())


# check that aa_star is now accepted along exactly one path that gives its expected value
list(fsa_reward_maxplus.compose(FST(''.join(aa_star))).iterate_paths())

Restricting to the legal paths `fsa_aa`, we can find the best legal path (which OpenFST calls the "shortest" path), giving the optimal action.  As we knew, the solution is rather boring in this case:

In [ ]:
fsa_reward_maxplus.compose(task.fsa_aa(xx=xx)).shortest_path()    # restrict to just the legal plans

This construction is manifestly correct, but there is a fly in the ointment.  Unfortunately, determinization blew up the machine exponentially!  If you look at `fsa_reward_det` or `fsa_reward_maxplus`, you'll see that it is a *trie* with a different path for every legal `aa` (and some illegal ones):

In [ ]:
print(fsa_reward_maxplus.num_states)
fsa_reward_maxplus

So searching for the best path in this machine is just as bad as the brute-force search over all `aa` values.
This would seem to dash our hopes of automatically finding the hand-crafted $O(J^2)$ algorithm.  That algorithm does correspond nicely to the *minimized* version of the DFA:

In [ ]:
fsa_reward_maxplus.compose(task.fsa_aa(xx=xx)).minimize()

You will notice that at time $j$, there are $j+1$ possible states, corresponding to the possible lengths $0, 1, \ldots, j$ of the longest `BI*` sequence ending at word $j$.  Thus, after reading a prefix of `aa`, the current state keeps track of which chunk (if any) `aa` has begun to propose.  This is necessary to determine how much expected reward might be achieved by future actions that would end the chunk in one place or another.  As for completed chunks before the currently proposed chunk, their expected reward was already accounted for on the path that read the prefix of `aa` (with different paths to the current state corresponding to different prefixes of different rewards).

In short, we would quite like to get the $O(J^2)$ machine above and run shortest-path on it, which would be an $O(J^2)$ algorithm.  The difficulty is to get it without first constructing an exponentially large DFA via determinization.  

In other words, if two states created by determinization would later be merged by minimization, we would like determinization to regard them as the same state in the first place.  It turns out that this can be done using a trick for "determinizing better" in the expectation semiring:

In [ ]:
def determinize_with_merging(self, *args, **kwargs):
    assert self.semiring is ExpectationSemiringWeight
    ExpectationSemiringWeight.aggressive_quantization = True     # temporarily modifies behavior of `quantize` on expectation semirings (WARNING: not thread-safe)
    result = self.push().determinize(*args, **kwargs)            # must push first for aggressive quantization to be correct
    ExpectationSemiringWeight.aggressive_quantization = False    
    return result

FST.determinize_with_merging = determinize_with_merging  # install method dynamically

# fsa_reward_det = fsa_reward.compose(task.fsa_aa(xx=xx)).determinize_with_merging()
fsa_reward_det = fsa_reward.compose(task.fsa_aa(xx=xx)).determinize_with_merging()
fsa_reward_det

The topology is identical to the previous minimized machine (except that it continues the pattern even at the final time step $j=5$ of having $j+1$ states, whereas minimization would notice that those states could actually be merged now that the string has ended).  We would complete the construction by again lowering this determinized FSA into the $(\max,+)$ semiring and finding the highest-weight path.

The explanation of the trick is a bit beyond the scope of this homework, but here is a quick sketch.  Determinization is a greedy algorithm that keeps adding states to the determinized machine.  When it creates a  creates a new edge to a state, it looks up a description of the target state in a hash table to determine whether that state has already been created and can be reused.  Reusing existing states is crucial to keeping the machine small: otherwise determinization would *always* create a trie (possibly an infinite one)!  

In the weighted case, the state description includes some "residual weights" that are elements of the weight semiring.  To allow hash lookup to succeed even if the residual weights are slightly different (e.g., due to floating-point error), OpenFST quantizes the weights before putting them into the state description.  For very interesting reasons, it turns out that in our Bayes agent construction (where the rewards do not affect the arc probabilities), it is safe to quantize the *second* element of the `ExpectationSemiringWeight` by simply making it `0`, provided that weight pushing is applied to the machine before determinization.  This allows states to merge during determinization.

### Putting it all together

You now know how to write the following agent!  Just generalize the example above.  Your solution should work whenever `model` is an `FstBoltzmannMachine` and `task` is an `FstTaskSetting`.  It will derive a Bayes decision rule that uses dynamic programming to the extent that is possible given the specific structure of the model and the reward function.

In [ ]:
class FstBayesAgent(BayesAgent):
    
    def decision(self, *, xx, oo=None):
        # Hint: Just copy and adjust the commands above.  Make sure to use `self.model` and
        #       `self.task`, not the global variables `model` and `task`.  Also make sure
        #       to consider `oo`.  
        #       For generality, you should call `remove_epsilon()` before determinizing, 
        #       in case there are any epsilons in the way (perhaps none for this task and model).
        ### STUDENTS START
        raise NotImplementedError()  # REPLACE ME
        ### STUDENTS END


In [ ]:
def test_bayes_agent(false_pos_penalty):
    task = IobTask(false_pos_penalty=false_pos_penalty)   # will have its own reward machine with different weights
    agent = FstBayesAgent(task, model)
    agent.test_F1(iterate_data('dev'))

In [ ]:
%time test_bayes_agent(1)   #  what we've been using so far (strongly discourages false positives)

In [ ]:
%time test_bayes_agent(0)    # doesn't mind false positives at all

Well, that was interesting.  Seems like we'd better try some in-between values of $\lambda$!  But first, let's pick an in-between value and confirm that our FST agent is doing the same thing as the brute force agent.

In [ ]:
def compare_agents(false_pos_penalty):
    task = IobTask(false_pos_penalty=false_pos_penalty)
    agent_old = BayesAgent(task, model)
    agent_new = FstBayesAgent(task, model)
    for xx,oo,yy in iterate_data('dev-small'):
        aa_old = agent_old.decision(xx=xx,oo=oo)
        aa_new = agent_new.decision(xx=xx,oo=oo)
        assert aa_old==aa_new, f'xx={xx},oo={oo},old={aa_old},new={aa_new}'
compare_agents(0.2)

If that worked, look for a value of $\lambda \geq 0$ that gets high F1 on dev data.  (Remember that $\lambda$ is tuning a proxy reward to balance precision and recall.)  I suggest trying golden section search, which is a charming little algorithm — it's quick to implement yourself, or grab some code from the web as long as you understand it.

# Studying and developing your system

Use the notebook to study what your system is doing.  What are the large parametersin your model?  Do they correspond to frequent words, or other indicators of named entities?  What kinds of errors are you making on training and development data?  If you wanted to add new feature templates, what might they be?  Consider playing with other chunking tasks — real or artificial — to get a sense of how your model behaves.

Can you improve performance on development data by changing the model, how you train the model, or how you decode?

There is a bunch of Python overhead in this implementation.  Discuss specific ways in which the system might be sped up, whether in Python or Cython or by switching to Java or C++.

Place your experiments and notes here.

<b>Answer:</b> <span style='color:red'>FILL IN</span>

# The Final Test!!!

Now select a particular decision agent for chunking.  It should use a particular model trained in a particular way on the training set, with hyperparameters perhaps informed by your experience on the dev set.  Measure its F1 reward on the **test** dataset, which you have not looked at until now.

In [ ]:
### final_agent = ????
### STUDENTS START
raise NotImplementedError()  # REPLACE ME
### STUDENTS END

final_agent.test_F1(iterate_data('test'))